In [1]:
import shared_functions
#下面用到什么定义就从shared_functions中载入
from shared_functions import read_from_files
import os
import pandas as pd
import numpy as np
import math
import sys
import time
import pickle
import json
import datetime
import random
import torch
import sklearn
from sklearn import *
from shared_functions import performance_assessment
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
sns.set_style('darkgrid', {'axes.facecolor': '0.9'})

import graphviz
import xgboost
# For imbalanced learning
#import imblearn
import warnings
warnings.filterwarnings('ignore')
from shared_functions import training_loop
import math
import torch.nn as nn
from shared_functions import prepare_generators
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [2]:
DIR_INPUT='D:/Credit Card Fraud Detection/simulated-data-transformed/data/'

BEGIN_DATE = "2018-06-11"
END_DATE = "2018-09-14"

print("Load  files")
%time transactions_df=read_from_files(DIR_INPUT, BEGIN_DATE, END_DATE)
print("{0} transactions loaded, containing {1} fraudulent transactions".format(len(transactions_df),transactions_df.TX_FRAUD.sum()))

output_feature="TX_FRAUD"

input_features=['TX_AMOUNT','TX_DURING_WEEKEND', 'TX_DURING_NIGHT', 'CUSTOMER_ID_NB_TX_1DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW', 'CUSTOMER_ID_NB_TX_7DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW', 'CUSTOMER_ID_NB_TX_30DAY_WINDOW',
       'CUSTOMER_ID_AVG_AMOUNT_30DAY_WINDOW', 'TERMINAL_ID_NB_TX_1DAY_WINDOW',
       'TERMINAL_ID_RISK_1DAY_WINDOW', 'TERMINAL_ID_NB_TX_7DAY_WINDOW',
       'TERMINAL_ID_RISK_7DAY_WINDOW', 'TERMINAL_ID_NB_TX_30DAY_WINDOW',
       'TERMINAL_ID_RISK_30DAY_WINDOW']

Load  files
CPU times: total: 734 ms
Wall time: 782 ms
919767 transactions loaded, containing 8195 fraudulent transactions


In [3]:
from shared_functions import get_train_test_set, scaleData
# Set the starting day for the training period, and the deltas
start_date_training = datetime.datetime.strptime("2018-07-25", "%Y-%m-%d")
delta_train=7
delta_delay=7
delta_test=7
delta_valid = delta_test
start_date_training_with_valid = start_date_training+datetime.timedelta(days=-(delta_delay+delta_valid))
(train_df, valid_df)=get_train_test_set(transactions_df,start_date_training_with_valid,
                                       delta_train=delta_train,delta_delay=delta_delay,delta_test=delta_test)
# By default, scales input data
(train_df, valid_df)=scaleData(train_df, valid_df,input_features)

In [4]:
if torch.cuda.is_available():
    DEVICE = "cuda" 
else:
    DEVICE = "cpu"
print("Selected device is",DEVICE)

SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

Selected device is cuda


In [5]:
x_train = torch.FloatTensor(train_df[input_features].values)
x_valid = torch.FloatTensor(valid_df[input_features].values)
y_train = torch.FloatTensor(train_df[output_feature].values)
y_valid = torch.FloatTensor(valid_df[output_feature].values)

In [6]:
# new Dataset: FraudDatasetUnsupervised, which only receives the descriptive features of the transaction x
# and returns it as both input and output.去掉标签的数据
class FraudDatasetUnsupervised(torch.utils.data.Dataset):
    
    def __init__(self, x,output=True):
        'Initialization'
        self.x = x
        self.output = output

    def __len__(self):
        'Returns the total number of samples'
        return len(self.x)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample index
        item = self.x[index].to(DEVICE)
        if self.output:
            return item, item
        else:
            return item

In [7]:
training_set = FraudDatasetUnsupervised(x_train)
valid_set = FraudDatasetUnsupervised(x_valid)
training_generator,valid_generator = prepare_generators(training_set, valid_set, batch_size = 64)

In [8]:
def per_sample_mse(model, generator):
    
    model.eval()
    criterion = torch.nn.MSELoss(reduction="none")
    batch_losses = []
    
    for x_batch, y_batch in generator:
        # Forward pass
        y_pred = model(x_batch)
        # Compute Loss
        loss = criterion(y_pred.squeeze(), y_batch)
        loss_app = list(torch.mean(loss,axis=1).detach().cpu().numpy())
        batch_losses.extend(loss_app)
    
    return batch_losses

In [9]:
from tqdm import tqdm
import optuna

class GELU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = input * (1 + torch.erf(input / torch.sqrt(torch.tensor(2.0)) / self.sigma)) / 2  # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.gelu1 = GELU(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.gelu2 = GELU(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.gelu3 = GELU(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.gelu1(hidden)
            
            code = self.fc2(hidden)
            code = self.gelu2(code)
 
            hidden = self.fc3(code)
            hidden = self.gelu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

In [27]:
best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=20)

[I 2024-12-03 14:57:32,856] A new study created in memory with name: no-name-0b4ac85c-daa8-452b-a93e-dee1bdfce4cb



Epoch 0: train loss: 0.5043518998683296
valid loss: 0.1705069158816598

Epoch 1: train loss: 0.11854787085768027
valid loss: 0.06667149253393131

Epoch 2: train loss: 0.048344600192461715
valid loss: 0.03007140745882128

Epoch 3: train loss: 0.02688401435318458
valid loss: 0.02167191175448764

Epoch 4: train loss: 0.019827575727328517
valid loss: 0.015809430936198743

Epoch 5: train loss: 0.01385420200817401
valid loss: 0.01052539900678103

Epoch 6: train loss: 0.008445383857999773
valid loss: 0.005971252251304482

Epoch 7: train loss: 0.006085145780033153
valid loss: 0.004990899246287802

Epoch 8: train loss: 0.005184439288489663
valid loss: 0.004239611204489659

Epoch 9: train loss: 0.004296266963530909
valid loss: 0.003538998474921685

Epoch 10: train loss: 0.0034186265325168393
valid loss: 0.0028726795027948547

Epoch 11: train loss: 0.002834448125102089
valid loss: 0.0025307620523110684

Epoch 12: train loss: 0.0025308152933847194
valid loss: 0.0022701204197145388

Epoch 13: trai

[I 2024-12-03 15:02:38,312] Trial 0 finished with value: 1.078 and parameters: {'sigma_1': 0.14552588024296545, 'sigma_2': 3.205164181715946, 'sigma_3': 3.1080087541916464}. Best is trial 0 with value: 1.078.


0.851
0.227
Updated Best Model!
Confusion Matrix:
[[52539  5592]
 [  115   259]]

Epoch 0: train loss: 0.5100161156423005
valid loss: 0.18631743718691862

Epoch 1: train loss: 0.13537306064672155
valid loss: 0.08576069586182553

Epoch 2: train loss: 0.06901931362743018
valid loss: 0.04214849866763816

Epoch 3: train loss: 0.035337643599917624
valid loss: 0.025354734893705023

Epoch 4: train loss: 0.02413255881952624
valid loss: 0.019735981344850988

Epoch 5: train loss: 0.01918766487667597
valid loss: 0.01509066897500408

Epoch 6: train loss: 0.013653182321767649
valid loss: 0.010111595990512865

Epoch 7: train loss: 0.009358499048234999
valid loss: 0.006357648417027921

Epoch 8: train loss: 0.0060037731054554165
valid loss: 0.004503896329399259

Epoch 9: train loss: 0.004735583234017441
valid loss: 0.0036386730307153337

Epoch 10: train loss: 0.004073142252566852
valid loss: 0.003147451924109231

Epoch 11: train loss: 0.00357803389001335
valid loss: 0.002781221791313035

Epoch 12: tra

[I 2024-12-03 15:07:39,417] Trial 1 finished with value: 1.09 and parameters: {'sigma_1': 2.189625271327292, 'sigma_2': 3.5083552374676574, 'sigma_3': 3.11379397477385}. Best is trial 1 with value: 1.09.


...

0.85
0.226

Epoch 0: train loss: 0.5235756852060386
valid loss: 0.19733124214931916

Epoch 1: train loss: 0.15291965138843822
valid loss: 0.08891574901228394

Epoch 2: train loss: 0.0696506790948558
valid loss: 0.04390825437473469

Epoch 3: train loss: 0.03554722154156855
valid loss: 0.02396262898593327

Epoch 4: train loss: 0.021548273467439198
valid loss: 0.015898322349648334

Epoch 5: train loss: 0.013465137484022211
valid loss: 0.008649133908679415

Epoch 6: train loss: 0.007491650068500288
valid loss: 0.0052961533665249905

Epoch 7: train loss: 0.005419276597514307
valid loss: 0.004097416932450333

Epoch 8: train loss: 0.004431547184811324
valid loss: 0.003381235378697677

Epoch 9: train loss: 0.003679394464780457
valid loss: 0.002717447976831123

Epoch 10: train loss: 0.0028965014587717403
valid loss: 0.002058307253475723

Epoch 11: train loss: 0.002001402418825412
valid loss: 0.0013728112392661878

Epoch 12: train loss: 0.001351132528527184
valid loss: 0.0010235778488666633

Epo

[I 2024-12-03 16:31:10,474] Trial 18 finished with value: 1.074 and parameters: {'sigma_1': 3.042130527859531, 'sigma_2': 2.6223375108832356, 'sigma_3': 2.5743189348242477}. Best is trial 9 with value: 1.093.


0.848
0.226

Epoch 0: train loss: 0.5139273208767232
valid loss: 0.2142406779478808

Epoch 1: train loss: 0.16390869611193984
valid loss: 0.0941314023537714

Epoch 2: train loss: 0.07061060176303977
valid loss: 0.04397370770695757

Epoch 3: train loss: 0.0389184501257157
valid loss: 0.028386448749183307

Epoch 4: train loss: 0.02587354596237693
valid loss: 0.01979564630574053

Epoch 5: train loss: 0.017856765624819533
valid loss: 0.013124493261687254

Epoch 6: train loss: 0.011049431554365895
valid loss: 0.007891007984067248

Epoch 7: train loss: 0.007663663897403368
valid loss: 0.0059879217650537775

Epoch 8: train loss: 0.006230211903316184
valid loss: 0.004885987581079571

Epoch 9: train loss: 0.005254609970955524
valid loss: 0.004098849219035947

Epoch 10: train loss: 0.004500038311932499
valid loss: 0.0035585107000626094

Epoch 11: train loss: 0.0038589565687720266
valid loss: 0.0029683646403731462

Epoch 12: train loss: 0.0031893203871050427
valid loss: 0.002366890944095196

Epoc

[I 2024-12-03 16:36:19,587] Trial 19 finished with value: 1.078 and parameters: {'sigma_1': 2.0920236420400364, 'sigma_2': 3.8386305884146266, 'sigma_3': 3.7504724673749488}. Best is trial 9 with value: 1.093.


0.852
0.226


In [17]:
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: GELU1={sigma1}, GELU2={sigma2}, GELU3={sigma3}")

Best trial parameters:
{'sigma_1': 4.154498525138923, 'sigma_2': 3.9751598714493444, 'sigma_3': 2.6531766740713265}
Optimal sigma values: GELU1=4.154498525138923, GELU2=3.9751598714493444, GELU3=2.6531766740713265


In [10]:
#实验结果：[I 2024-12-02 21:42:22,265] Trial 4 finished with value: 1.1 and parameters: {'sigma_1': 4.108071217704759, 'sigma_2': 3.05488181077701, 'sigma_3': 2.3962617153103962}. Best is trial 4 with value: 1.1.
#0.869
#0.231
#'sigma_1': 4.108071217704759, 
#'sigma_2': 3.05488181077701, 
#'sigma_3': 2.3962617153103962

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)

[I 2025-03-02 09:06:01,628] A new study created in memory with name: no-name-b98fe2c6-3ef7-46dd-a960-00b0a3fb6bde



Epoch 0: train loss: 0.5123742886357736
valid loss: 0.18337174157301586

Epoch 1: train loss: 0.13761347607063062
valid loss: 0.07784592379800609

Epoch 2: train loss: 0.0574941604325181
valid loss: 0.0347329105815438

Epoch 3: train loss: 0.033132810743942316
valid loss: 0.025019208981039747

Epoch 4: train loss: 0.02110654040262901
valid loss: 0.013666522870339006

Epoch 5: train loss: 0.010117714874932533
valid loss: 0.0055019977932561775

Epoch 6: train loss: 0.004542241587163791
valid loss: 0.003049991508218068

Epoch 7: train loss: 0.003046221282986664
valid loss: 0.0020960292663130035

Epoch 8: train loss: 0.0019621309890731436
valid loss: 0.0012734530223253604

Epoch 9: train loss: 0.001228153009414379
valid loss: 0.0009673513278372769

Epoch 10: train loss: 0.0009200444958810686
valid loss: 0.0007301643785010372

Epoch 11: train loss: 0.0007474764521112647
valid loss: 0.0006042572559631058

Epoch 12: train loss: 0.0006316633035335779
valid loss: 0.00048659503118954393

Epoch 

[I 2025-03-02 09:16:17,621] Trial 0 finished with value: 1.092 and parameters: {'sigma_1': 4.7607666409078515, 'sigma_2': 1.0306519030280907, 'sigma_3': 4.254113588226719}. Best is trial 0 with value: 1.092.


0.863
0.229
Updated Best Model!
Confusion Matrix:
[[52552  5579]
 [  102   272]]

Epoch 0: train loss: 0.5135413723682583
valid loss: 0.19214718882503407

Epoch 1: train loss: 0.141505044737179
valid loss: 0.08883772993999753

Epoch 2: train loss: 0.07173970378899723
valid loss: 0.04484826132980852

Epoch 3: train loss: 0.03731301529813793
valid loss: 0.02628908527740201

Epoch 4: train loss: 0.024626479382639177
valid loss: 0.02008642358043806

Epoch 5: train loss: 0.019820367712634356
valid loss: 0.016234527322682525

Epoch 6: train loss: 0.014671200474574592
valid loss: 0.010477690253651044

Epoch 7: train loss: 0.00952864617236621
valid loss: 0.006306604656792892

Epoch 8: train loss: 0.005904347971250903
valid loss: 0.00430743637001091

Epoch 9: train loss: 0.004608059733911074
valid loss: 0.003532024443052872

Epoch 10: train loss: 0.003948262441136748
valid loss: 0.003030163323750994

Epoch 11: train loss: 0.0034071883774561946
valid loss: 0.002591564152084413

Epoch 12: train l

[I 2025-03-02 09:27:12,774] Trial 1 finished with value: 1.081 and parameters: {'sigma_1': 4.92587809533317, 'sigma_2': 3.7972482191739054, 'sigma_3': 1.9037497945201067}. Best is trial 0 with value: 1.092.


...

0.84
0.229

Epoch 0: train loss: 0.5178176315633106
valid loss: 0.17249950926974822

Epoch 1: train loss: 0.120961279225452
valid loss: 0.07149017154763306

Epoch 2: train loss: 0.05484075719458594
valid loss: 0.03395248034694156

Epoch 3: train loss: 0.028608762652009553
valid loss: 0.02069855592737758

Epoch 4: train loss: 0.0185233290266267
valid loss: 0.013250124660101744

Epoch 5: train loss: 0.01048495979484591
valid loss: 0.006517601602766765

Epoch 6: train loss: 0.006513171553937629
valid loss: 0.004982645754941581

Epoch 7: train loss: 0.005340100821850191
valid loss: 0.004133849395082785

Epoch 8: train loss: 0.004583912610653325
valid loss: 0.0036137848787131856

Epoch 9: train loss: 0.0039842744119933686
valid loss: 0.0030977309051687355

Epoch 10: train loss: 0.003405782912304023
valid loss: 0.0025904574522448327

Epoch 11: train loss: 0.002695170914853102
valid loss: 0.0019341837639584526

Epoch 12: train loss: 0.0018806858820730311
valid loss: 0.0015408126548914838

Epo

[I 2025-03-02 10:46:00,417] Trial 8 finished with value: 1.077 and parameters: {'sigma_1': 3.7434347285298033, 'sigma_2': 4.133759694546447, 'sigma_3': 0.21890694389331372}. Best is trial 0 with value: 1.092.


0.854
0.223

Epoch 0: train loss: 0.5046405407455865
valid loss: 0.2112785721167189

Epoch 1: train loss: 0.13381084117034084
valid loss: 0.07550822032279655

Epoch 2: train loss: 0.061304887284811434
valid loss: 0.03808175789528206

Epoch 3: train loss: 0.029916072217124313
valid loss: 0.02114098947291654

Epoch 4: train loss: 0.01931903852117791
valid loss: 0.014188964535670529

Epoch 5: train loss: 0.012008148196543323
valid loss: 0.007545514266347624

Epoch 6: train loss: 0.006324244473694003
valid loss: 0.004098310949830491

Epoch 7: train loss: 0.004295977126218442
valid loss: 0.003152251348683173

Epoch 8: train loss: 0.003389048392761563
valid loss: 0.0025092986734237908

Epoch 9: train loss: 0.0025180487369986723
valid loss: 0.0017116851664448535

Epoch 10: train loss: 0.001598631982554915
valid loss: 0.0010528226645123925

Epoch 11: train loss: 0.0010508253697577925
valid loss: 0.0007810470393332118

Epoch 12: train loss: 0.0008248839366880119
valid loss: 0.000666824838097583

[I 2025-03-02 10:57:15,159] Trial 9 finished with value: 1.083 and parameters: {'sigma_1': 3.6796514454817895, 'sigma_2': 1.6907690118477698, 'sigma_3': 3.216056358685279}. Best is trial 0 with value: 1.092.


0.853
0.23


In [13]:
#0.86,0.862,0.864,0.869,0.863; 0.23,0.23,0.229,0.231,0.229; 请给出这两组数据的均值和标准差
#第一组数据的均值约为 0.8636，标准差约为 0.00288，总和为 4.318。
#第二组数据的均值约为 0.23，标准差约为 0.001，总和为 1.15。
#两组数据相加后的均值约为 1.0934，标准差约为 0.00288，总和为 5.467

In [30]:
class SiLU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = input * torch.sigmoid(input / self.sigma) # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.silu1 = SiLU(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.silu2 = SiLU(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.silu3 = SiLU(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.silu1(hidden)
            
            code = self.fc2(hidden)
            code = self.silu2(code)
 
            hidden = self.fc3(code)
            hidden = self.silu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2024-12-03 16:40:25,533] A new study created in memory with name: no-name-d21cbee2-61f4-4e83-a824-4529d8edfe82



Epoch 0: train loss: 0.5186284254118088
valid loss: 0.19544177768660373

Epoch 1: train loss: 0.14958315862638316
valid loss: 0.08783522057728689

Epoch 2: train loss: 0.06400370962889312
valid loss: 0.040033551462279646

Epoch 3: train loss: 0.03564840887030355
valid loss: 0.026923045368427462

Epoch 4: train loss: 0.025361228793018976
valid loss: 0.01981965399301443

Epoch 5: train loss: 0.018033098359992925
valid loss: 0.012378497870099512

Epoch 6: train loss: 0.009803588134165405
valid loss: 0.00604965213299449

Epoch 7: train loss: 0.006159807161390639
valid loss: 0.004686248142022864

Epoch 8: train loss: 0.005104218545747319
valid loss: 0.003964578307264286

Epoch 9: train loss: 0.004480979399778747
valid loss: 0.0035394821730773142

Epoch 10: train loss: 0.004038532590181932
valid loss: 0.003179285231636725

Epoch 11: train loss: 0.0036960203493509023
valid loss: 0.002949897694029932

Epoch 12: train loss: 0.0033915769288772237
valid loss: 0.002642244034156692

Epoch 13: trai

[I 2024-12-03 16:44:41,965] Trial 0 finished with value: 1.071 and parameters: {'sigma_1': 0.5938746851891842, 'sigma_2': 4.593083040719566, 'sigma_3': 1.9238539990433898}. Best is trial 0 with value: 1.071.


0.842
0.229
Updated Best Model!
Confusion Matrix:
[[52541  5590]
 [  113   261]]

Epoch 0: train loss: 0.5137471224592703
valid loss: 0.20596642149764982

Epoch 1: train loss: 0.14696430596549698
valid loss: 0.09430717821818232

Epoch 2: train loss: 0.06777768532779374
valid loss: 0.03924704181854842

Epoch 3: train loss: 0.0348251954817977
valid loss: 0.025650239758234206

Epoch 4: train loss: 0.02258116255975533
valid loss: 0.0160290422793376

Epoch 5: train loss: 0.01437582666353531
valid loss: 0.010615266062054641

Epoch 6: train loss: 0.009687539117489131
valid loss: 0.006520124790487719

Epoch 7: train loss: 0.006243562757217434
valid loss: 0.004570119480648246

Epoch 8: train loss: 0.004862579898588203
valid loss: 0.0036987498808458774

Epoch 9: train loss: 0.003985481673271091
valid loss: 0.003079815643020178

Epoch 10: train loss: 0.00315499121897959
valid loss: 0.0022844470843362314

Epoch 11: train loss: 0.002228877993178577
valid loss: 0.0015906154227066439

Epoch 12: train

[I 2024-12-03 16:48:50,306] Trial 1 finished with value: 1.08 and parameters: {'sigma_1': 1.692766487265329, 'sigma_2': 3.895310850172154, 'sigma_3': 1.1147221505432356}. Best is trial 1 with value: 1.08.


...

0.848
0.229

Epoch 0: train loss: 0.49277501445028565
valid loss: 0.16832245744642665

Epoch 1: train loss: 0.1131709270628016
valid loss: 0.05685712383255932

Epoch 2: train loss: 0.04215664542271234
valid loss: 0.02503033932196638

Epoch 3: train loss: 0.02134808688647119
valid loss: 0.01567669936298021

Epoch 4: train loss: 0.01331857297200902
valid loss: 0.00910092431369945

Epoch 5: train loss: 0.007311970536173259
valid loss: 0.004975790648003582

Epoch 6: train loss: 0.005203355244631535
valid loss: 0.003961101322831799

Epoch 7: train loss: 0.004371208733862113
valid loss: 0.0034644036827344306

Epoch 8: train loss: 0.0036928988637021576
valid loss: 0.0029010103147761907

Epoch 9: train loss: 0.0029482287805514982
valid loss: 0.002164085433421443

Epoch 10: train loss: 0.0021279747826585915
valid loss: 0.0015436741969385311

Epoch 11: train loss: 0.001529491000566596
valid loss: 0.0011943819814951978

Epoch 12: train loss: 0.0012568152418589828
valid loss: 0.001036365679178967


[I 2024-12-03 17:18:32,492] Trial 8 finished with value: 1.079 and parameters: {'sigma_1': 0.5190629567692205, 'sigma_2': 0.7500168767080223, 'sigma_3': 4.018114731826644}. Best is trial 6 with value: 1.091.


0.848
0.231

Epoch 0: train loss: 0.5224251648265135
valid loss: 0.19607449546863473

Epoch 1: train loss: 0.1447618489062866
valid loss: 0.08462191391023782

Epoch 2: train loss: 0.0629626552451466
valid loss: 0.035387741493397074

Epoch 3: train loss: 0.02939738475327624
valid loss: 0.021292895414545886

Epoch 4: train loss: 0.01913722195212033
valid loss: 0.01387747174098355

Epoch 5: train loss: 0.01188048152350972
valid loss: 0.007870656399493987

Epoch 6: train loss: 0.007213252371294605
valid loss: 0.005164600491951235

Epoch 7: train loss: 0.005462408037304195
valid loss: 0.0041144904837042825

Epoch 8: train loss: 0.004600781478262889
valid loss: 0.003551350903310533

Epoch 9: train loss: 0.0040506010795640715
valid loss: 0.003089223498773705

Epoch 10: train loss: 0.0035784287933780176
valid loss: 0.0030432348179025323

Epoch 11: train loss: 0.0030592761925011197
valid loss: 0.0022214195791560687

Epoch 12: train loss: 0.002261947664716254
valid loss: 0.0015274437928770777

E

[I 2024-12-03 17:22:42,547] Trial 9 finished with value: 1.083 and parameters: {'sigma_1': 1.1458862371478755, 'sigma_2': 2.5243479611951534, 'sigma_3': 0.8757207872180713}. Best is trial 6 with value: 1.091.


0.854
0.229
Best trial parameters:
{'sigma_1': 0.32487841546726154, 'sigma_2': 3.034287558059582, 'sigma_3': 2.931114456661248}
Optimal sigma values: SiLU1=0.32487841546726154, SiLU2=3.034287558059582, SiLU3=2.931114456661248


In [10]:
import optuna

class SiLU(nn.Module):  # 激活函数含噪声参数
    def __init__(self,sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = input * torch.sigmoid(input / self.sigma) # 前向传播
        return temp

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.silu1 = SiLU(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.silu2 = SiLU(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.silu3 = SiLU(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.silu1(hidden)
            
            code = self.fc2(hidden)
            code = self.silu2(code)
 
            hidden = self.fc3(code)
            hidden = self.silu3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-02 12:16:41,542] A new study created in memory with name: no-name-e438ba57-a28c-446e-8fcf-3d07ba5321fe



Epoch 0: train loss: 0.520786259864701
valid loss: 0.1892861767870481

Epoch 1: train loss: 0.1483831764314587
valid loss: 0.09340558383064192

Epoch 2: train loss: 0.06719742869392009
valid loss: 0.03823695424638811

Epoch 3: train loss: 0.03231048354091939
valid loss: 0.023172924795844516

Epoch 4: train loss: 0.021393750449660347
valid loss: 0.01683867254950961

Epoch 5: train loss: 0.015311928240699876
valid loss: 0.012475786223763325

Epoch 6: train loss: 0.012334658469591668
valid loss: 0.010837596676775503

Epoch 7: train loss: 0.010372110760466234
valid loss: 0.00827249868014634

Epoch 8: train loss: 0.006411395269182526
valid loss: 0.003896677263890256

Epoch 9: train loss: 0.0037954010121234076
valid loss: 0.0027807572885808316

Epoch 10: train loss: 0.002731536325520478
valid loss: 0.0021015394149510994

Epoch 11: train loss: 0.0019409505430146889
valid loss: 0.0014740927989055183

Epoch 12: train loss: 0.0014811294803522299
valid loss: 0.0011299547527807313

Epoch 13: trai

[I 2025-03-02 12:26:50,456] Trial 0 finished with value: 1.075 and parameters: {'sigma_1': 4.936201430577657, 'sigma_2': 4.942415234911959, 'sigma_3': 0.5769408886651006}. Best is trial 0 with value: 1.075.


0.849
0.226
Updated Best Model!
Confusion Matrix:
[[52536  5595]
 [  118   256]]

Epoch 0: train loss: 0.5166274642511262
valid loss: 0.19065224204232784

Epoch 1: train loss: 0.14259408081010924
valid loss: 0.08781393640191178

Epoch 2: train loss: 0.07315001740204045
valid loss: 0.04698521891211877

Epoch 3: train loss: 0.04073832466589362
valid loss: 0.029302344394878287

Epoch 4: train loss: 0.027109422290922065
valid loss: 0.02079549702647768

Epoch 5: train loss: 0.020377077416020017
valid loss: 0.016155920897310223

Epoch 6: train loss: 0.014625781530916804
valid loss: 0.010764903989442385

Epoch 7: train loss: 0.010467265980553244
valid loss: 0.007194070042177139

Epoch 8: train loss: 0.006375381246255443
valid loss: 0.00443240967566463

Epoch 9: train loss: 0.004769712505460562
valid loss: 0.0036679418209197173

Epoch 10: train loss: 0.004152946487108306
valid loss: 0.0031982966923903003

Epoch 11: train loss: 0.0037034612693681414
valid loss: 0.0028532076401819213

Epoch 12: 

[I 2025-03-02 12:37:58,177] Trial 1 finished with value: 1.085 and parameters: {'sigma_1': 2.206257909603749, 'sigma_2': 4.029325362152258, 'sigma_3': 3.2590582846892184}. Best is trial 1 with value: 1.085.


...

0.842
0.227

Epoch 0: train loss: 0.5261885410826476
valid loss: 0.20759150942818064

Epoch 1: train loss: 0.13610135203309315
valid loss: 0.07928136361182714

Epoch 2: train loss: 0.06602373505883759
valid loss: 0.04432130357240393

Epoch 3: train loss: 0.036922875193462316
valid loss: 0.02626436189672009

Epoch 4: train loss: 0.02472643524027264
valid loss: 0.019785120573322303

Epoch 5: train loss: 0.01853926511424421
valid loss: 0.013820191611194872

Epoch 6: train loss: 0.010984507932489734
valid loss: 0.006576402919478918

Epoch 7: train loss: 0.006335317162773705
valid loss: 0.004665691026008072

Epoch 8: train loss: 0.005041738067011048
valid loss: 0.003838537526164265

Epoch 9: train loss: 0.004268786431549306
valid loss: 0.0032749955644169465

Epoch 10: train loss: 0.0036649797718311316
valid loss: 0.002800286660044673

Epoch 11: train loss: 0.003074373490084497
valid loss: 0.0022691755649084074

Epoch 12: train loss: 0.002338834189447925
valid loss: 0.0017915977678475443

Ep

[I 2025-03-02 13:56:57,263] Trial 8 finished with value: 1.072 and parameters: {'sigma_1': 2.489728174893267, 'sigma_2': 3.0582962945720067, 'sigma_3': 1.19486490117434}. Best is trial 4 with value: 1.089.


0.849
0.223

Epoch 0: train loss: 0.5103828883934659
valid loss: 0.21881603128760238

Epoch 1: train loss: 0.14571085626376976
valid loss: 0.08617476667862772

Epoch 2: train loss: 0.0712139149077137
valid loss: 0.05064329149537399

Epoch 3: train loss: 0.03934063005437416
valid loss: 0.02609435564181844

Epoch 4: train loss: 0.02426777991161719
valid loss: 0.019086633020085716

Epoch 5: train loss: 0.017691109984771244
valid loss: 0.012918953858621296

Epoch 6: train loss: 0.012467811423234542
valid loss: 0.009177812933921814

Epoch 7: train loss: 0.0082719389920837
valid loss: 0.005343927007392456

Epoch 8: train loss: 0.005309186457236961
valid loss: 0.00383404517136616

Epoch 9: train loss: 0.004176888715392993
valid loss: 0.003121459880803045

Epoch 10: train loss: 0.0033212135544573858
valid loss: 0.0024069692648163504

Epoch 11: train loss: 0.0024034753486328895
valid loss: 0.0015991668999983924

Epoch 12: train loss: 0.0015391297751937924
valid loss: 0.0010845758184050134

Epoc

[I 2025-03-02 14:07:15,483] Trial 9 finished with value: 1.078 and parameters: {'sigma_1': 3.88305241993981, 'sigma_2': 2.2763183027092957, 'sigma_3': 1.2215097318416799}. Best is trial 4 with value: 1.089.


0.848
0.23
Best trial parameters:
{'sigma_1': 4.454554072434889, 'sigma_2': 1.25727828829437, 'sigma_3': 3.1100036683720114}
Optimal sigma values: SiLU1=4.454554072434889, SiLU2=1.25727828829437, SiLU3=3.1100036683720114


In [ ]:
#MiSH

In [ ]:
import optuna

# class SiLU(nn.Module):  # 激活函数含噪声参数
#     def __init__(self,sigma):
#         super().__init__()
#         self.sigma = sigma

#     def forward(self, input):
#         temp = input * torch.sigmoid(input / self.sigma) # 前向传播
#         return temp

class PMish(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        # Original Mish: x * tanh(softplus(x))
        # Parameterized version: x * tanh(softplus(x / sigma))
        softplus = torch.nn.functional.softplus(input / self.sigma)
        output = input * torch.tanh(softplus)
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.PMish1 = PMish(trial.suggest_float("sigma_1", 0.01, 5.0))
            self.PMish2 = PMish(trial.suggest_float("sigma_2", 0.01, 5.0))
            self.PMish3 = PMish(trial.suggest_float("sigma_3", 0.01, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.PMish1(hidden)
            
            code = self.fc2(hidden)
            code = self.PMish2(code)
 
            hidden = self.fc3(code)
            hidden = self.PMish3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-07 06:32:27,044] A new study created in memory with name: no-name-b1346940-30c1-4ff6-8a0a-8031812aa808



Epoch 0: train loss: 0.45518349805565456
valid loss: 0.14420220368221157

Epoch 1: train loss: 0.1042436251174623
valid loss: 0.0520004616286911

Epoch 2: train loss: 0.03848124666755617
valid loss: 0.025209232574440743

Epoch 3: train loss: 0.021287489580934293
valid loss: 0.014905587272321592

Epoch 4: train loss: 0.012540812395338547
valid loss: 0.00893131933872247

Epoch 5: train loss: 0.006120343406125547
valid loss: 0.0033670512932642807

Epoch 6: train loss: 0.0033061828491648387
valid loss: 0.002214770790957973

Epoch 7: train loss: 0.0020895312397440726
valid loss: 0.0013634681793901805

Epoch 8: train loss: 0.0013335537358557085
valid loss: 0.0009505687525452629

Epoch 9: train loss: 0.0010197148778534585
valid loss: 0.0008615662186989265

Epoch 10: train loss: 0.0008417863402017936
valid loss: 0.0007158023835467734

Epoch 11: train loss: 0.0007108238758407996
valid loss: 0.000564697130597222

Epoch 12: train loss: 0.0006192317523110531
valid loss: 0.00045742688263522184

Ep

[I 2025-03-07 06:41:52,742] Trial 0 finished with value: 1.071 and parameters: {'sigma_1': 3.435277104647319, 'sigma_2': 1.456448875767326, 'sigma_3': 1.195330529018753}. Best is trial 0 with value: 1.071.


Confusion Matrix:
[[52544  5587]
 [  110   264]]

Epoch 0: train loss: 0.447509842246315
valid loss: 0.13822620931218882

Epoch 1: train loss: 0.08854096589421223
valid loss: 0.03966591948327797

Epoch 2: train loss: 0.029926562749461965
valid loss: 0.01941068478518985

Epoch 3: train loss: 0.015933161405543753
valid loss: 0.009719784131344876

Epoch 4: train loss: 0.0070631358566396575
valid loss: 0.003954499379904748

Epoch 5: train loss: 0.004092916857850085
valid loss: 0.0029161716962762462

Epoch 6: train loss: 0.0029359051932573645
valid loss: 0.0019234767628965972

Epoch 7: train loss: 0.0017552750765984382
valid loss: 0.0011538404133552532

Epoch 8: train loss: 0.0011269831813746764
valid loss: 0.0009318427544702965

Epoch 9: train loss: 0.000905572769245895
valid loss: 0.0007117110760335409

Epoch 10: train loss: 0.0007772886958723375
valid loss: 0.0006487812488746977

Epoch 11: train loss: 0.0006885321875226134
valid loss: 0.0005216584357056421

Epoch 12: train loss: 0.000610

[I 2025-03-07 06:52:02,696] Trial 1 finished with value: 1.07 and parameters: {'sigma_1': 1.8352764115064717, 'sigma_2': 0.8035703058711101, 'sigma_3': 1.1599212699678378}. Best is trial 0 with value: 1.071.


...

0.849
0.224

Epoch 0: train loss: 0.44656264410135166
valid loss: 0.166578779430663

Epoch 1: train loss: 0.09929300557081205
valid loss: 0.047157656112329555

Epoch 2: train loss: 0.03834881336761222
valid loss: 0.02625063805856177

Epoch 3: train loss: 0.01986925031795713
valid loss: 0.011288004368543625

Epoch 4: train loss: 0.00910863733403456
valid loss: 0.004942109117976548

Epoch 5: train loss: 0.004608818562590049
valid loss: 0.003264629757388042

Epoch 6: train loss: 0.0036057969953697424
valid loss: 0.0026877238144182034

Epoch 7: train loss: 0.0029887579100778107
valid loss: 0.0021745947975909613

Epoch 8: train loss: 0.002328124907600959
valid loss: 0.0016011568619055379

Epoch 9: train loss: 0.001537457073161229
valid loss: 0.0009671492144895483

Epoch 10: train loss: 0.0009153977608369186
valid loss: 0.0006073060874570202

Epoch 11: train loss: 0.0006578893913495903
valid loss: 0.0004893616048455025

Epoch 12: train loss: 0.0005474625337214732
valid loss: 0.00040689760906

[I 2025-03-07 07:56:11,821] Trial 7 finished with value: 1.089 and parameters: {'sigma_1': 2.3352176641768847, 'sigma_2': 0.88782596447751, 'sigma_3': 4.388846799415538}. Best is trial 7 with value: 1.089.


Confusion Matrix:
[[52550  5581]
 [  104   270]]

Epoch 0: train loss: 0.4470782012644849
valid loss: 0.11989837602758016

Epoch 1: train loss: 0.07867843772411119
valid loss: 0.0442314512478035

Epoch 2: train loss: 0.033419706003943436
valid loss: 0.0216894713306769

Epoch 3: train loss: 0.018116085200675588
valid loss: 0.012210859361364216

Epoch 4: train loss: 0.010015116070965252
valid loss: 0.0069728466934051185

Epoch 5: train loss: 0.00703967119349847
valid loss: 0.0055792185386306925

Epoch 6: train loss: 0.005878724870325403
valid loss: 0.004791053650452872

Epoch 7: train loss: 0.005018581432578854
valid loss: 0.004019106151612445

Epoch 8: train loss: 0.004159526524835588
valid loss: 0.003331587301780943

Epoch 9: train loss: 0.0032468944489484008
valid loss: 0.002595385483751084

Epoch 10: train loss: 0.002526013747887796
valid loss: 0.0021159462661283915

Epoch 11: train loss: 0.002111353425097155
valid loss: 0.0017879309284098731

Epoch 12: train loss: 0.0018344322435627

[I 2025-03-07 08:07:39,138] Trial 8 finished with value: 1.069 and parameters: {'sigma_1': 0.19306043941470555, 'sigma_2': 0.5857500401114696, 'sigma_3': 1.7888078757620693}. Best is trial 7 with value: 1.089.


0.845
0.224

Epoch 0: train loss: 0.45436441549322576
valid loss: 0.14548286245983155

Epoch 1: train loss: 0.09079868289469305
valid loss: 0.05176072500320406

Epoch 2: train loss: 0.03548399898816563
valid loss: 0.021230995635720874

Epoch 3: train loss: 0.01781279943486316
valid loss: 0.011165095544089385

Epoch 4: train loss: 0.008069081842520026
valid loss: 0.004261723593961386

Epoch 5: train loss: 0.004135624474639262
valid loss: 0.0028728966206780387

Epoch 6: train loss: 0.0029886634765508057
valid loss: 0.0019990169011880874

Epoch 7: train loss: 0.0018548091151106062
valid loss: 0.0011109814712589434

Epoch 8: train loss: 0.0010336612823800349
valid loss: 0.0008483714840115338

Epoch 9: train loss: 0.000750753191587102
valid loss: 0.0005543352677575897

Epoch 10: train loss: 0.0006048439217381442
valid loss: 0.00047784737210155876

Epoch 11: train loss: 0.0005130613074472341
valid loss: 0.0003903983310825583

Epoch 12: train loss: 0.0004339717313189622
valid loss: 0.00035425

In [9]:
import optuna

# class SiLU(nn.Module):  # 激活函数含噪声参数
#     def __init__(self,sigma):
#         super().__init__()
#         self.sigma = sigma

#     def forward(self, input):
#         temp = input * torch.sigmoid(input / self.sigma) # 前向传播
#         return temp

class PMish(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        # Original Mish: x * tanh(softplus(x))
        # Parameterized version: x * tanh(softplus(x / sigma))
        softplus = torch.nn.functional.softplus(input / self.sigma)
        output = input * torch.tanh(softplus)
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.PMish1 = PMish(trial.suggest_float("sigma_1", 0.01, 5.0))
            self.PMish2 = PMish(trial.suggest_float("sigma_2", 0.01, 5.0))
            self.PMish3 = PMish(trial.suggest_float("sigma_3", 0.01, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.PMish1(hidden)
            
            code = self.fc2(hidden)
            code = self.PMish2(code)
 
            hidden = self.fc3(code)
            hidden = self.PMish3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-07 09:13:46,925] A new study created in memory with name: no-name-f18391e1-be80-4312-9fcf-5573cbd3b558



Epoch 0: train loss: 0.4482217215884826
valid loss: 0.13418430482429233

Epoch 1: train loss: 0.08874515999984672
valid loss: 0.04000123525699957

Epoch 2: train loss: 0.03045867356680063
valid loss: 0.020638181020816167

Epoch 3: train loss: 0.016836623143917574
valid loss: 0.01087844353118311

Epoch 4: train loss: 0.007539379103714689
valid loss: 0.003906273624294849

Epoch 5: train loss: 0.003762813069969843
valid loss: 0.0025760292137917027

Epoch 6: train loss: 0.002458504473934766
valid loss: 0.0015756098484834198

Epoch 7: train loss: 0.001491151523378359
valid loss: 0.0010400294018031448

Epoch 8: train loss: 0.0011096692701766488
valid loss: 0.0008480605276497438

Epoch 9: train loss: 0.0009227068792609231
valid loss: 0.0007610183639217759

Epoch 10: train loss: 0.0008010719371932856
valid loss: 0.000680236261442324

Epoch 11: train loss: 0.0007039881542573877
valid loss: 0.0005701615291907162

Epoch 12: train loss: 0.0006304957292657693
valid loss: 0.00048236064231175867

Ep

[I 2025-03-07 09:24:09,536] Trial 0 finished with value: 1.087 and parameters: {'sigma_1': 1.3874435218016146, 'sigma_2': 0.6016216005969973, 'sigma_3': 1.7080481675850172}. Best is trial 0 with value: 1.087.


0.857
0.23
Updated Best Model!
Confusion Matrix:
[[52543  5588]
 [  111   263]]

Epoch 0: train loss: 0.44852188701810847
valid loss: 0.14070864498126703

Epoch 1: train loss: 0.09431745217361596
valid loss: 0.046357687298065976

Epoch 2: train loss: 0.034719340125700715
valid loss: 0.022741404463806764

Epoch 3: train loss: 0.020083178722443876
valid loss: 0.014300195189813772

Epoch 4: train loss: 0.011444927541357439
valid loss: 0.006639711413267857

Epoch 5: train loss: 0.00626788502554614
valid loss: 0.004651237341773803

Epoch 6: train loss: 0.00500600476171394
valid loss: 0.003908568179830298

Epoch 7: train loss: 0.004167474776036311
valid loss: 0.0031622086445510875

Epoch 8: train loss: 0.0032156019037087266
valid loss: 0.0023480922908679955

Epoch 9: train loss: 0.0022107840326025256
valid loss: 0.001654905533161055

Epoch 10: train loss: 0.001697851840208995
valid loss: 0.0013664023250295613

Epoch 11: train loss: 0.001456591201780086
valid loss: 0.0011927721366385517

Epoc

[I 2025-03-07 09:34:19,248] Trial 1 finished with value: 1.079 and parameters: {'sigma_1': 0.6541423949677474, 'sigma_2': 2.4018455859893875, 'sigma_3': 0.8917124127028049}. Best is trial 0 with value: 1.087.


...

0.846
0.229

Epoch 0: train loss: 0.4759493685090633
valid loss: 0.14339247341038752

Epoch 1: train loss: 0.10216334548709384
valid loss: 0.058341128293608054

Epoch 2: train loss: 0.042833650967259364
valid loss: 0.026794172290081533

Epoch 3: train loss: 0.02305563767022145
valid loss: 0.01649367887771032

Epoch 4: train loss: 0.012922752675302967
valid loss: 0.0075746527414952145

Epoch 5: train loss: 0.00707630420492759
valid loss: 0.00525008025121127

Epoch 6: train loss: 0.005546320487752229
valid loss: 0.00429027709636654

Epoch 7: train loss: 0.004660035992605722
valid loss: 0.0035997873323567495

Epoch 8: train loss: 0.003909419230399137
valid loss: 0.0029790736770491455

Epoch 9: train loss: 0.0030954452358403495
valid loss: 0.002268868887827533

Epoch 10: train loss: 0.0021828017431270354
valid loss: 0.0015302572430549138

Epoch 11: train loss: 0.0015575964058670135
valid loss: 0.0011847236803147554

Epoch 12: train loss: 0.0012720809680392812
valid loss: 0.0012151001546089

[I 2025-03-07 10:47:24,876] Trial 8 finished with value: 1.072 and parameters: {'sigma_1': 4.602528638313524, 'sigma_2': 4.37127185820264, 'sigma_3': 0.15581176083270765}. Best is trial 5 with value: 1.088.


0.851
0.221

Epoch 0: train loss: 0.4437044216088424
valid loss: 0.1593660553296407

Epoch 1: train loss: 0.09772621558170702
valid loss: 0.056536334512826525

Epoch 2: train loss: 0.04405402138928952
valid loss: 0.0281077077826027

Epoch 3: train loss: 0.024524356532364684
valid loss: 0.01736508716671356

Epoch 4: train loss: 0.015336057041677332
valid loss: 0.010543626417594204

Epoch 5: train loss: 0.009160176594430015
valid loss: 0.005703575463833216

Epoch 6: train loss: 0.005540657956126444
valid loss: 0.0039425431353757616

Epoch 7: train loss: 0.004190349195688521
valid loss: 0.00306202897349057

Epoch 8: train loss: 0.003180156734569023
valid loss: 0.0022653235584405846

Epoch 9: train loss: 0.0021792504130453087
valid loss: 0.001477297688032821

Epoch 10: train loss: 0.0014692216562454449
valid loss: 0.0011261438487779572

Epoch 11: train loss: 0.001150075086467253
valid loss: 0.0008941018086211766

Epoch 12: train loss: 0.0009624284341137582
valid loss: 0.0007856788642353332

[I 2025-03-07 10:57:36,647] Trial 9 finished with value: 1.077 and parameters: {'sigma_1': 2.2557446440389533, 'sigma_2': 2.6603744303107066, 'sigma_3': 4.0711762251492525}. Best is trial 5 with value: 1.088.


0.846
0.231
Best trial parameters:
{'sigma_1': 1.1611961565460787, 'sigma_2': 2.435688247625747, 'sigma_3': 2.8133295330470096}
Optimal sigma values: SiLU1=1.1611961565460787, SiLU2=2.435688247625747, SiLU3=2.8133295330470096


In [10]:
import optuna

# class SiLU(nn.Module):  # 激活函数含噪声参数
#     def __init__(self,sigma):
#         super().__init__()
#         self.sigma = sigma

#     def forward(self, input):
#         temp = input * torch.sigmoid(input / self.sigma) # 前向传播
#         return temp

class PMish(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        # Original Mish: x * tanh(softplus(x))
        # Parameterized version: x * tanh(softplus(x / sigma))
        softplus = torch.nn.functional.softplus(input / self.sigma)
        output = input * torch.tanh(softplus)
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.PMish1 = PMish(trial.suggest_float("sigma_1", 0.01, 5.0))
            self.PMish2 = PMish(trial.suggest_float("sigma_2", 0.01, 5.0))
            self.PMish3 = PMish(trial.suggest_float("sigma_3", 0.01, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.PMish1(hidden)
            
            code = self.fc2(hidden)
            code = self.PMish2(code)
 
            hidden = self.fc3(code)
            hidden = self.PMish3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-07 10:57:36,679] A new study created in memory with name: no-name-d1c7c03c-cd88-4f85-97d1-2f0484cc40b0



Epoch 0: train loss: 0.4529342468521677
valid loss: 0.1532416801309325

Epoch 1: train loss: 0.10599643081412374
valid loss: 0.052888381153426534

Epoch 2: train loss: 0.03651488973825002
valid loss: 0.022378162051362744

Epoch 3: train loss: 0.01947854849622081
valid loss: 0.014170927806793014

Epoch 4: train loss: 0.01188145705764084
valid loss: 0.0078352805639266

Epoch 5: train loss: 0.005752208072159717
valid loss: 0.0030829936765958783

Epoch 6: train loss: 0.0028318149245448276
valid loss: 0.0017862996207982749

Epoch 7: train loss: 0.001634757705148573
valid loss: 0.001095412991867182

Epoch 8: train loss: 0.0010919201185396583
valid loss: 0.0008244152675456586

Epoch 9: train loss: 0.0008584501230234483
valid loss: 0.0006378132889532011

Epoch 10: train loss: 0.0007035447613374655
valid loss: 0.0005598632173957507

Epoch 11: train loss: 0.0006002118482215937
valid loss: 0.000494933342293943

Epoch 12: train loss: 0.0005243437762969186
valid loss: 0.00039518948009363607

Epoch

[I 2025-03-07 11:08:02,039] Trial 0 finished with value: 1.091 and parameters: {'sigma_1': 1.5166668071409213, 'sigma_2': 0.24841225703746808, 'sigma_3': 3.676669271836084}. Best is trial 0 with value: 1.091.


0.862
0.229
Updated Best Model!
Confusion Matrix:
[[52548  5583]
 [  106   268]]

Epoch 0: train loss: 0.4703644303316482
valid loss: 0.17079529383632003

Epoch 1: train loss: 0.10599906925631983
valid loss: 0.060521735682513546

Epoch 2: train loss: 0.04716390432134612
valid loss: 0.02830989548193464

Epoch 3: train loss: 0.02371380525689223
valid loss: 0.01627744033199842

Epoch 4: train loss: 0.013122593909217113
valid loss: 0.008055978796134393

Epoch 5: train loss: 0.00674745849851006
valid loss: 0.004746091468615936

Epoch 6: train loss: 0.004989000750940656
valid loss: 0.0038148975101569313

Epoch 7: train loss: 0.004214811967961832
valid loss: 0.0032725898093477787

Epoch 8: train loss: 0.0036490891035512647
valid loss: 0.0027977134004398122

Epoch 9: train loss: 0.0031006868699846885
valid loss: 0.002299459507851734

Epoch 10: train loss: 0.0023586791686835674
valid loss: 0.0015984819025752933

Epoch 11: train loss: 0.0015204305234467557
valid loss: 0.0010702595459500866

Epoc

[I 2025-03-07 11:18:44,808] Trial 1 finished with value: 1.084 and parameters: {'sigma_1': 1.137951121295139, 'sigma_2': 2.4046328498839564, 'sigma_3': 2.61286355814698}. Best is trial 0 with value: 1.091.


...

0.851
0.227

Epoch 0: train loss: 0.46904188138833464
valid loss: 0.1730681962084249

Epoch 1: train loss: 0.11955692474489914
valid loss: 0.05932001487804892

Epoch 2: train loss: 0.04339840086479716
valid loss: 0.026917357965693122

Epoch 3: train loss: 0.021873271827324226
valid loss: 0.013751459100329485

Epoch 4: train loss: 0.01111177370276295
valid loss: 0.006239908411233721

Epoch 5: train loss: 0.005734504328063025
valid loss: 0.004100109770883873

Epoch 6: train loss: 0.004371638962821804
valid loss: 0.003284171743628879

Epoch 7: train loss: 0.003503802144075486
valid loss: 0.002553542830111062

Epoch 8: train loss: 0.0026213171577791597
valid loss: 0.001791049925422123

Epoch 9: train loss: 0.0016684623003423564
valid loss: 0.0010780389962687356

Epoch 10: train loss: 0.001064875717835775
valid loss: 0.000803426597960792

Epoch 11: train loss: 0.0008245268760101663
valid loss: 0.0006719800335249503

Epoch 12: train loss: 0.000685610571038334
valid loss: 0.000527400917645869

[I 2025-03-07 12:30:52,356] Trial 8 finished with value: 1.082 and parameters: {'sigma_1': 2.3686052416013172, 'sigma_2': 1.651700605894983, 'sigma_3': 4.821425439185221}. Best is trial 0 with value: 1.091.


0.851
0.231

Epoch 0: train loss: 0.4465697445046605
valid loss: 0.14929222819583662

Epoch 1: train loss: 0.09606684989742864
valid loss: 0.043226439196392484

Epoch 2: train loss: 0.034997418896788396
valid loss: 0.02346479767027611

Epoch 3: train loss: 0.019404645015113412
valid loss: 0.011830306634347622

Epoch 4: train loss: 0.009191725523937601
valid loss: 0.005302192720386093

Epoch 5: train loss: 0.005204769481110598
valid loss: 0.003915371500398414

Epoch 6: train loss: 0.004225523974105402
valid loss: 0.003344289591927794

Epoch 7: train loss: 0.003471507814051281
valid loss: 0.0026167377841002384

Epoch 8: train loss: 0.002553301661871427
valid loss: 0.001799189649417976

Epoch 9: train loss: 0.0016855342600456337
valid loss: 0.001218049890518219

Epoch 10: train loss: 0.0012356631287690127
valid loss: 0.0009637421163694699

Epoch 11: train loss: 0.0010321632069751467
valid loss: 0.0008258169111375864

Epoch 12: train loss: 0.0008971777760005693
valid loss: 0.00077154740173

[I 2025-03-07 12:41:02,170] Trial 9 finished with value: 1.078 and parameters: {'sigma_1': 0.8250606287140755, 'sigma_2': 0.47258955757751303, 'sigma_3': 3.0681434171936157}. Best is trial 0 with value: 1.091.


0.851
0.227
Best trial parameters:
{'sigma_1': 1.5166668071409213, 'sigma_2': 0.24841225703746808, 'sigma_3': 3.676669271836084}
Optimal sigma values: SiLU1=1.5166668071409213, SiLU2=0.24841225703746808, SiLU3=3.676669271836084


In [11]:
import optuna

# class SiLU(nn.Module):  # 激活函数含噪声参数
#     def __init__(self,sigma):
#         super().__init__()
#         self.sigma = sigma

#     def forward(self, input):
#         temp = input * torch.sigmoid(input / self.sigma) # 前向传播
#         return temp

class PMish(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        # Original Mish: x * tanh(softplus(x))
        # Parameterized version: x * tanh(softplus(x / sigma))
        softplus = torch.nn.functional.softplus(input / self.sigma)
        output = input * torch.tanh(softplus)
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.PMish1 = PMish(trial.suggest_float("sigma_1", 0.01, 5.0))
            self.PMish2 = PMish(trial.suggest_float("sigma_2", 0.01, 5.0))
            self.PMish3 = PMish(trial.suggest_float("sigma_3", 0.01, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.PMish1(hidden)
            
            code = self.fc2(hidden)
            code = self.PMish2(code)
 
            hidden = self.fc3(code)
            hidden = self.PMish3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-07 12:41:02,216] A new study created in memory with name: no-name-61610f1d-182c-4125-a841-5a603f57b557



Epoch 0: train loss: 0.46098818931992147
valid loss: 0.15167671445288944

Epoch 1: train loss: 0.10152038032577211
valid loss: 0.05614208691025692

Epoch 2: train loss: 0.04182478583936026
valid loss: 0.02617408866622572

Epoch 3: train loss: 0.021168802364464015
valid loss: 0.014090534697587047

Epoch 4: train loss: 0.010256677237161244
valid loss: 0.005316480117610814

Epoch 5: train loss: 0.004945542467677927
valid loss: 0.0036255424996186557

Epoch 6: train loss: 0.003908347134655573
valid loss: 0.0029075995459134788

Epoch 7: train loss: 0.0032524030141881446
valid loss: 0.002372437685694993

Epoch 8: train loss: 0.0025557130048194827
valid loss: 0.001757815053248829

Epoch 9: train loss: 0.0016220371137830797
valid loss: 0.000993106869695698

Epoch 10: train loss: 0.0008859419041085297
valid loss: 0.0006107203135345782

Epoch 11: train loss: 0.0006311387677014099
valid loss: 0.0005095307953475053

Epoch 12: train loss: 0.0005150363966644552
valid loss: 0.0003974102413640587

Epo

[I 2025-03-07 12:51:07,814] Trial 0 finished with value: 1.081 and parameters: {'sigma_1': 4.8539514761956015, 'sigma_2': 1.7261764427171729, 'sigma_3': 4.499082234489828}. Best is trial 0 with value: 1.081.


0.845
0.236
Updated Best Model!
Confusion Matrix:
[[52538  5593]
 [  116   258]]

Epoch 0: train loss: 0.45474563641598303
valid loss: 0.1448627870916669

Epoch 1: train loss: 0.10492301800325655
valid loss: 0.05633334435441129

Epoch 2: train loss: 0.04504022092147095
valid loss: 0.027391558973108486

Epoch 3: train loss: 0.02392285052743913
valid loss: 0.01695158459510797

Epoch 4: train loss: 0.014550182757361384
valid loss: 0.009863219794924142

Epoch 5: train loss: 0.008664322054515776
valid loss: 0.006101578012200408

Epoch 6: train loss: 0.00613164991170638
valid loss: 0.004623869697110412

Epoch 7: train loss: 0.004991599383993683
valid loss: 0.0037766510281009506

Epoch 8: train loss: 0.004173155390434002
valid loss: 0.0031698795788682225

Epoch 9: train loss: 0.0034223294424705107
valid loss: 0.002499521981105412

Epoch 10: train loss: 0.002527087569827958
valid loss: 0.0017181536608345074

Epoch 11: train loss: 0.0016585973863370617
valid loss: 0.00118645010283853

Epoch 12:

[I 2025-03-07 13:01:14,341] Trial 1 finished with value: 1.088 and parameters: {'sigma_1': 4.048255424622733, 'sigma_2': 4.20591870935168, 'sigma_3': 2.8993534297264087}. Best is trial 1 with value: 1.088.


...

0.856
0.227

Epoch 0: train loss: 0.4612431288848188
valid loss: 0.15982305521522064

Epoch 1: train loss: 0.1148131879805151
valid loss: 0.05750881136481553

Epoch 2: train loss: 0.04071771445725092
valid loss: 0.024902982057118026

Epoch 3: train loss: 0.021362371505888083
valid loss: 0.014856337743704436

Epoch 4: train loss: 0.012636823349064269
valid loss: 0.008314752349212685

Epoch 5: train loss: 0.007588227201577269
valid loss: 0.005422949401376759

Epoch 6: train loss: 0.005655007699529425
valid loss: 0.0042546540446569275

Epoch 7: train loss: 0.004645789016851083
valid loss: 0.0035594162627891764

Epoch 8: train loss: 0.003910029790142505
valid loss: 0.002936587572642322

Epoch 9: train loss: 0.00326894576430684
valid loss: 0.002391268020620458

Epoch 10: train loss: 0.002508326895865763
valid loss: 0.001810741192874972

Epoch 11: train loss: 0.0017355708022518208
valid loss: 0.0011862405489295558

Epoch 12: train loss: 0.0012565190995884009
valid loss: 0.0009572716152468503

[I 2025-03-07 14:11:49,234] Trial 8 finished with value: 1.074 and parameters: {'sigma_1': 3.478606063105709, 'sigma_2': 4.035330251943707, 'sigma_3': 2.154789444591714}. Best is trial 1 with value: 1.088.


0.848
0.226

Epoch 0: train loss: 0.4365587648837676
valid loss: 0.13575092288640028

Epoch 1: train loss: 0.08030422301482174
valid loss: 0.03421260420579077

Epoch 2: train loss: 0.026052979087318164
valid loss: 0.014599843640796474

Epoch 3: train loss: 0.009043579678279108
valid loss: 0.003964891568426796

Epoch 4: train loss: 0.0035921685519159892
valid loss: 0.0023023724131021506

Epoch 5: train loss: 0.002097689436517574
valid loss: 0.0013842729853511956

Epoch 6: train loss: 0.0013185140904841365
valid loss: 0.000996982508855541

Epoch 7: train loss: 0.000986257527684873
valid loss: 0.0007414079027611742

Epoch 8: train loss: 0.0007846753736116513
valid loss: 0.0006240191761292314

Epoch 9: train loss: 0.000656036980106027
valid loss: 0.0004977005836522064

Epoch 10: train loss: 0.0005525727848001279
valid loss: 0.00043931311205388276

Epoch 11: train loss: 0.00047953223016506584
valid loss: 0.00039717234269387593

Epoch 12: train loss: 0.0004191894162428231
valid loss: 0.00034

[I 2025-03-07 14:21:52,753] Trial 9 finished with value: 1.0819999999999999 and parameters: {'sigma_1': 3.074144740492197, 'sigma_2': 0.48394677853033513, 'sigma_3': 0.6243951132517639}. Best is trial 1 with value: 1.088.


0.843
0.239
Best trial parameters:
{'sigma_1': 4.048255424622733, 'sigma_2': 4.20591870935168, 'sigma_3': 2.8993534297264087}
Optimal sigma values: SiLU1=4.048255424622733, SiLU2=4.20591870935168, SiLU3=2.8993534297264087


In [12]:
import optuna

# class SiLU(nn.Module):  # 激活函数含噪声参数
#     def __init__(self,sigma):
#         super().__init__()
#         self.sigma = sigma

#     def forward(self, input):
#         temp = input * torch.sigmoid(input / self.sigma) # 前向传播
#         return temp

class PMish(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        # Original Mish: x * tanh(softplus(x))
        # Parameterized version: x * tanh(softplus(x / sigma))
        softplus = torch.nn.functional.softplus(input / self.sigma)
        output = input * torch.tanh(softplus)
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.PMish1 = PMish(trial.suggest_float("sigma_1", 0.01, 5.0))
            self.PMish2 = PMish(trial.suggest_float("sigma_2", 0.01, 5.0))
            self.PMish3 = PMish(trial.suggest_float("sigma_3", 0.01, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.PMish1(hidden)
            
            code = self.fc2(hidden)
            code = self.PMish2(code)
 
            hidden = self.fc3(code)
            hidden = self.PMish3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_models = None
best_model_states = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_states = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)


best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

# 分别提取 GELU1、GELU2 和 GELU3 的 sigma 值
sigma1 = best_trial.params.get("sigma_1")  # GELU1 的 sigma
sigma2 = best_trial.params.get("sigma_2")  # GELU2 的 sigma
sigma3 = best_trial.params.get("sigma_3")  # GELU3 的 sigma

# 打印最优的 sigma 值
print(f"Optimal sigma values: SiLU1={sigma1}, SiLU2={sigma2}, SiLU3={sigma3}")

[I 2025-03-07 14:21:52,793] A new study created in memory with name: no-name-9f0c6c43-c543-4751-8327-b707b87598a0



Epoch 0: train loss: 0.46498034032592356
valid loss: 0.14373027978699063

Epoch 1: train loss: 0.09777769495165028
valid loss: 0.05675749277156559

Epoch 2: train loss: 0.04307752537468081
valid loss: 0.024962165895747682

Epoch 3: train loss: 0.01984610278417586
valid loss: 0.012377945184096939

Epoch 4: train loss: 0.010880170522674378
valid loss: 0.006724410938719908

Epoch 5: train loss: 0.006182408087395388
valid loss: 0.004372428397296882

Epoch 6: train loss: 0.004697250281318125
valid loss: 0.0035429867660077136

Epoch 7: train loss: 0.003876175364589828
valid loss: 0.0028945752409352955

Epoch 8: train loss: 0.00312000479287402
valid loss: 0.0022771211998801823

Epoch 9: train loss: 0.002180111235792056
valid loss: 0.0014946720630307096

Epoch 10: train loss: 0.0014294590103176438
valid loss: 0.0010341279004456201

Epoch 11: train loss: 0.0011199029369758759
valid loss: 0.0009183441749977487

Epoch 12: train loss: 0.0009615325017691542
valid loss: 0.0008088805179030232

Epoch

[I 2025-03-07 14:31:59,662] Trial 0 finished with value: 1.057 and parameters: {'sigma_1': 1.2429483401428678, 'sigma_2': 3.514266464025207, 'sigma_3': 0.6451317715366279}. Best is trial 0 with value: 1.057.


0.836
0.221
Updated Best Model!
Confusion Matrix:
[[52540  5591]
 [  114   260]]

Epoch 0: train loss: 0.46328600356444344
valid loss: 0.14970546273585877

Epoch 1: train loss: 0.09191054900142989
valid loss: 0.04185809294586299

Epoch 2: train loss: 0.034051610363458586
valid loss: 0.0223746469559891

Epoch 3: train loss: 0.017460140492232875
valid loss: 0.010673501498264367

Epoch 4: train loss: 0.007402857557209796
valid loss: 0.004302239643481097

Epoch 5: train loss: 0.0042534397548175
valid loss: 0.0030049187374235095

Epoch 6: train loss: 0.0029253832201017547
valid loss: 0.0019349706936831195

Epoch 7: train loss: 0.0017409681699356247
valid loss: 0.0011523284435353645

Epoch 8: train loss: 0.0011341782344859556
valid loss: 0.0008753333983769834

Epoch 9: train loss: 0.0008978077877314234
valid loss: 0.0008000676135934604

Epoch 10: train loss: 0.0007572344536683834
valid loss: 0.0006300424557539273

Epoch 11: train loss: 0.0006479618267443772
valid loss: 0.0005120745434138232


[I 2025-03-07 14:42:05,429] Trial 1 finished with value: 1.074 and parameters: {'sigma_1': 1.950957698585951, 'sigma_2': 1.2576173296705022, 'sigma_3': 1.6340948784457747}. Best is trial 1 with value: 1.074.


...

0.855
0.227

Epoch 0: train loss: 0.4607350341690772
valid loss: 0.15889289245579413

Epoch 1: train loss: 0.10794037579421674
valid loss: 0.051074871201007094

Epoch 2: train loss: 0.0410078327825033
valid loss: 0.028829303346167172

Epoch 3: train loss: 0.024601264643067954
valid loss: 0.015270385896288307

Epoch 4: train loss: 0.011508715487696074
valid loss: 0.006951321664780297

Epoch 5: train loss: 0.006708075797245847
valid loss: 0.004956078889163954

Epoch 6: train loss: 0.005227815210472002
valid loss: 0.0039472317239116934

Epoch 7: train loss: 0.004276462527210994
valid loss: 0.003269258973600923

Epoch 8: train loss: 0.0033957318707331873
valid loss: 0.002501752505160339

Epoch 9: train loss: 0.002390581935550216
valid loss: 0.001661803555875874

Epoch 10: train loss: 0.0015993576518430893
valid loss: 0.0012627480340716964

Epoch 11: train loss: 0.0012686676380823743
valid loss: 0.0010121672192227116

Epoch 12: train loss: 0.0010863680020887334
valid loss: 0.000902717904272

[I 2025-03-07 15:52:47,106] Trial 8 finished with value: 1.072 and parameters: {'sigma_1': 2.2855430394813925, 'sigma_2': 4.084891309688508, 'sigma_3': 0.5790105859634521}. Best is trial 4 with value: 1.085.


0.846
0.226

Epoch 0: train loss: 0.464305840233202
valid loss: 0.1671275228424802

Epoch 1: train loss: 0.110842020840279
valid loss: 0.05974083027593751

Epoch 2: train loss: 0.0445978782793247
valid loss: 0.02695676947490928

Epoch 3: train loss: 0.023934306691705213
valid loss: 0.018035015685717918

Epoch 4: train loss: 0.014148421343825544
valid loss: 0.007590509931266797

Epoch 5: train loss: 0.006483812739647013
valid loss: 0.004574394002149666

Epoch 6: train loss: 0.004797629012711111
valid loss: 0.003579344790138793

Epoch 7: train loss: 0.0038698015251084736
valid loss: 0.00287691423735617

Epoch 8: train loss: 0.0030251893006313243
valid loss: 0.0021117215357145773

Epoch 9: train loss: 0.001977580126646967
valid loss: 0.001259084506713546

Epoch 10: train loss: 0.0010942632855422774
valid loss: 0.0007724525549454115

Epoch 11: train loss: 0.0007752677653397542
valid loss: 0.000675945515271093

Epoch 12: train loss: 0.0006384562428061936
valid loss: 0.00048288014956592213



[I 2025-03-07 16:02:56,270] Trial 9 finished with value: 1.078 and parameters: {'sigma_1': 4.609495673556397, 'sigma_2': 2.1589442934819227, 'sigma_3': 3.6527592889314158}. Best is trial 4 with value: 1.085.


0.845
0.233
Best trial parameters:
{'sigma_1': 3.54158522122103, 'sigma_2': 1.6201434727135235, 'sigma_3': 4.203304229284746}
Optimal sigma values: SiLU1=3.54158522122103, SiLU2=1.6201434727135235, SiLU3=4.203304229284746


In [17]:
##Gumbel

In [10]:
import optuna
import numpy as np
import torch
from torch import nn

# Gumbel activation class
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.1, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.1, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.1, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-09 11:24:28,549] A new study created in memory with name: no-name-9f4857a0-259b-4c1c-9600-d673c2fbe8ea



Epoch 0: train loss: 0.44043304275429274
valid loss: 0.1386297807830279

Epoch 1: train loss: 0.1029634215507521
valid loss: 0.05351090134411562

Epoch 2: train loss: 0.03906569771386726
valid loss: 0.025370978986427106

Epoch 3: train loss: 0.022157057919441572
valid loss: 0.016440019215065275

Epoch 4: train loss: 0.014308561902158462
valid loss: 0.011200225038728752

Epoch 5: train loss: 0.00870233932180583
valid loss: 0.004930872351816687

Epoch 6: train loss: 0.004633047082862081
valid loss: 0.0033788212543210943

Epoch 7: train loss: 0.0033857754910681715
valid loss: 0.002466735971806561

Epoch 8: train loss: 0.0023604654332504468
valid loss: 0.0016817776509815104

Epoch 9: train loss: 0.001672195999811441
valid loss: 0.001280173018086152

Epoch 10: train loss: 0.001301677239647876
valid loss: 0.0010427208017122734

Epoch 11: train loss: 0.00105239671628177
valid loss: 0.0008640167360636788

Epoch 12: train loss: 0.0008871905882337556
valid loss: 0.000691233031706798

Epoch 13: 

[I 2025-03-09 11:35:56,700] Trial 0 finished with value: 1.084 and parameters: {'sigma_1': 4.6982088930943995, 'sigma_2': 3.0642023033129724, 'sigma_3': 3.721936704065066}. Best is trial 0 with value: 1.084.


0.861
0.223
Updated Best Model!
Confusion Matrix:
[[52547  5584]
 [  107   267]]

Epoch 0: train loss: 0.4305816246036362
valid loss: 0.1332502772003575

Epoch 1: train loss: 0.0859365886786016
valid loss: 0.03938129469141608

Epoch 2: train loss: 0.029809310262128852
valid loss: 0.019040134443285686

Epoch 3: train loss: 0.015498562436817966
valid loss: 0.009714233826418391

Epoch 4: train loss: 0.007225014689604593
valid loss: 0.0039028237778642545

Epoch 5: train loss: 0.004075026667826448
valid loss: 0.002927644049896365

Epoch 6: train loss: 0.0030075302442142218
valid loss: 0.0019887316956749704

Epoch 7: train loss: 0.0017994275603961578
valid loss: 0.0010758625484030959

Epoch 8: train loss: 0.0010339842166464035
valid loss: 0.0007682751454832796

Epoch 9: train loss: 0.0007974382938552704
valid loss: 0.0006156679740636177

Epoch 10: train loss: 0.0006756936745033875
valid loss: 0.0005466646043605379

Epoch 11: train loss: 0.0005865498348210455
valid loss: 0.0004388215050357012

[I 2025-03-09 11:48:05,201] Trial 1 finished with value: 1.097 and parameters: {'sigma_1': 3.052745311719547, 'sigma_2': 0.828317193847135, 'sigma_3': 2.209002749440913}. Best is trial 1 with value: 1.097.


...

0.847
0.227

Epoch 0: train loss: 0.43554338000102205
valid loss: 0.12208401262760163

Epoch 1: train loss: 0.08585164643090439
valid loss: 0.047275644483787765

Epoch 2: train loss: 0.03803456732807378
valid loss: 0.02537298269997366

Epoch 3: train loss: 0.0219246623138354
valid loss: 0.015091497396103671

Epoch 4: train loss: 0.011390969936956739
valid loss: 0.006536824116228871

Epoch 5: train loss: 0.006353551373206236
valid loss: 0.0047188540984230126

Epoch 6: train loss: 0.004995923606749182
valid loss: 0.003882954178042099

Epoch 7: train loss: 0.004116341928207096
valid loss: 0.003129863887745378

Epoch 8: train loss: 0.0033476059053477833
valid loss: 0.0024923303224975903

Epoch 9: train loss: 0.0025033724419042526
valid loss: 0.0017560016119577847

Epoch 10: train loss: 0.0016774515997877533
valid loss: 0.0012217692654233813

Epoch 11: train loss: 0.0012344306522219515
valid loss: 0.0009543339618484241

Epoch 12: train loss: 0.0010046903949932518
valid loss: 0.0009249155571

[I 2025-03-09 13:16:08,598] Trial 8 finished with value: 1.079 and parameters: {'sigma_1': 2.960219015551997, 'sigma_2': 4.566062549129509, 'sigma_3': 2.5498604221242314}. Best is trial 1 with value: 1.097.


0.85
0.229

Epoch 0: train loss: 0.43321314693408985
valid loss: 0.1347176738938347

Epoch 1: train loss: 0.08167563221928438
valid loss: 0.042372767374815185

Epoch 2: train loss: 0.030190824517663247
valid loss: 0.01858808225926643

Epoch 3: train loss: 0.01483834803818474
valid loss: 0.008275694964972677

Epoch 4: train loss: 0.006170696734066119
valid loss: 0.0036097615499232636

Epoch 5: train loss: 0.0035947688692071285
valid loss: 0.002434643982436325

Epoch 6: train loss: 0.0023229639557297973
valid loss: 0.0014400278314785227

Epoch 7: train loss: 0.0013135112529069836
valid loss: 0.0008872933866127898

Epoch 8: train loss: 0.00089239929887857
valid loss: 0.0007625829689255585

Epoch 9: train loss: 0.000718166961245164
valid loss: 0.0005487786496059379

Epoch 10: train loss: 0.000589658797350307
valid loss: 0.0004669467831263319

Epoch 11: train loss: 0.0005070629445515417
valid loss: 0.00039150475058704617

Epoch 12: train loss: 0.0004339058784212397
valid loss: 0.00035916369

[I 2025-03-09 13:28:38,771] Trial 9 finished with value: 1.067 and parameters: {'sigma_1': 4.04053610943497, 'sigma_2': 0.5207405625034441, 'sigma_3': 3.7398078182307946}. Best is trial 1 with value: 1.097.


0.844
0.223
Best trial parameters:
{'sigma_1': 3.052745311719547, 'sigma_2': 0.828317193847135, 'sigma_3': 2.209002749440913}


In [13]:

# Gumbel activation class
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.5, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.5, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.5, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-09 15:10:02,184] A new study created in memory with name: no-name-d0f3d329-b301-42d5-89c8-83488947776e



Epoch 0: train loss: 0.4297480865614934
valid loss: 0.1163713755991941

Epoch 1: train loss: 0.07819963654905672
valid loss: 0.041295612314359735

Epoch 2: train loss: 0.03230882188337312
valid loss: 0.021693718751963697

Epoch 3: train loss: 0.019085419659874977
valid loss: 0.013821818433254142

Epoch 4: train loss: 0.011912931344003539
valid loss: 0.007741237023611212

Epoch 5: train loss: 0.006286560675533521
valid loss: 0.004378829624814292

Epoch 6: train loss: 0.0046705124324568715
valid loss: 0.0036055534814682898

Epoch 7: train loss: 0.0040273946699023915
valid loss: 0.003229943052560213

Epoch 8: train loss: 0.003491352297464496
valid loss: 0.0026838507828979546

Epoch 9: train loss: 0.002846586030736979
valid loss: 0.0020532133752388542

Epoch 10: train loss: 0.0019568655435436622
valid loss: 0.0013026545367196018

Epoch 11: train loss: 0.0012443838566757585
valid loss: 0.0009451235160091638

Epoch 12: train loss: 0.0009781547569027384
valid loss: 0.0007787149108480663

Epo

[I 2025-03-09 15:21:35,786] Trial 0 finished with value: 1.084 and parameters: {'sigma_1': 1.384923170274982, 'sigma_2': 4.335985255223839, 'sigma_3': 3.886580113570057}. Best is trial 0 with value: 1.084.


0.861
0.223
Updated Best Model!
Confusion Matrix:
[[52544  5587]
 [  110   264]]

Epoch 0: train loss: 0.4464187958699795
valid loss: 0.15880149330109194

Epoch 1: train loss: 0.09752014321034656
valid loss: 0.04935608935730705

Epoch 2: train loss: 0.0352896052603227
valid loss: 0.022167450824721914

Epoch 3: train loss: 0.019196907281839995
valid loss: 0.012963258427735724

Epoch 4: train loss: 0.010737604191829495
valid loss: 0.006334126043728874

Epoch 5: train loss: 0.005892065782402686
valid loss: 0.004279353574065929

Epoch 6: train loss: 0.0043982858161921
valid loss: 0.0031999933707675403

Epoch 7: train loss: 0.003192465348359746
valid loss: 0.0022651770503942146

Epoch 8: train loss: 0.002176367511249808
valid loss: 0.001606924793634645

Epoch 9: train loss: 0.0016280318683046264
valid loss: 0.0012703820420212819

Epoch 10: train loss: 0.0013460083239703808
valid loss: 0.0010827841139538468

Epoch 11: train loss: 0.0011543352775681714
valid loss: 0.000964855523608228

Epoch 

[I 2025-03-09 15:33:18,470] Trial 1 finished with value: 1.07 and parameters: {'sigma_1': 4.522547871349434, 'sigma_2': 4.544545008460191, 'sigma_3': 0.7906092774561136}. Best is trial 0 with value: 1.084.


...

0.838
0.244

Epoch 0: train loss: 0.43539053440293435
valid loss: 0.12978060766647423

Epoch 1: train loss: 0.08828444304532349
valid loss: 0.054061757644913236

Epoch 2: train loss: 0.040468088336944236
valid loss: 0.02355037301351301

Epoch 3: train loss: 0.020417422180849326
valid loss: 0.014346743633594018

Epoch 4: train loss: 0.011392884780020662
valid loss: 0.007231657104719369

Epoch 5: train loss: 0.006965654233495489
valid loss: 0.005202769029559822

Epoch 6: train loss: 0.005515162948559948
valid loss: 0.004163908829169521

Epoch 7: train loss: 0.004532938105135223
valid loss: 0.003448921862017628

Epoch 8: train loss: 0.0036624478929485576
valid loss: 0.0027280630711771425

Epoch 9: train loss: 0.002702187930957644
valid loss: 0.0018866882822349098

Epoch 10: train loss: 0.0018390102035287647
valid loss: 0.001362222743658298

Epoch 11: train loss: 0.0014053576423714905
valid loss: 0.0011096084830758308

Epoch 12: train loss: 0.001171119424038572
valid loss: 0.00095844478018

[I 2025-03-09 16:57:47,301] Trial 8 finished with value: 1.08 and parameters: {'sigma_1': 2.4924504045912843, 'sigma_2': 3.8987861620334847, 'sigma_3': 2.1996204083826307}. Best is trial 6 with value: 1.086.


0.854
0.226

Epoch 0: train loss: 0.4413388474037725
valid loss: 0.14849992171337045

Epoch 1: train loss: 0.1014532486144204
valid loss: 0.05537387283895511

Epoch 2: train loss: 0.04204265622290325
valid loss: 0.027391505147752866

Epoch 3: train loss: 0.02248361496073167
valid loss: 0.01473316867006281

Epoch 4: train loss: 0.01221969846928097
valid loss: 0.0077745140978018115

Epoch 5: train loss: 0.007065073091734421
valid loss: 0.005193529627667588

Epoch 6: train loss: 0.005587108722615482
valid loss: 0.004368121106981661

Epoch 7: train loss: 0.004800106161256208
valid loss: 0.0037385621332039275

Epoch 8: train loss: 0.004153917556389076
valid loss: 0.0032835574393145373

Epoch 9: train loss: 0.003423979139511062
valid loss: 0.00251551082757266

Epoch 10: train loss: 0.0024907578256543918
valid loss: 0.0017739931082027286

Epoch 11: train loss: 0.0017658041352846875
valid loss: 0.0013919918212652858

Epoch 12: train loss: 0.0014546672583434794
valid loss: 0.0011745230014540275

[I 2025-03-09 17:09:50,454] Trial 9 finished with value: 1.075 and parameters: {'sigma_1': 0.6228137416023212, 'sigma_2': 4.127044543927459, 'sigma_3': 4.87435570190993}. Best is trial 6 with value: 1.086.


0.849
0.226
Best trial parameters:
{'sigma_1': 2.6467672067042916, 'sigma_2': 2.016688944526466, 'sigma_3': 1.7004015254712996}


In [14]:
# Gumbel activation class
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.5, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.5, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.5, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-09 17:09:50,492] A new study created in memory with name: no-name-df018e58-5ae8-4c80-869b-dacf3fffb3dd



Epoch 0: train loss: 0.4275712137596671
valid loss: 0.1398944093229992

Epoch 1: train loss: 0.09453905169383746
valid loss: 0.05140976257052252

Epoch 2: train loss: 0.03982861860909893
valid loss: 0.025765490272372474

Epoch 3: train loss: 0.022671420007844386
valid loss: 0.016004695739251017

Epoch 4: train loss: 0.013538043071243407
valid loss: 0.008412125579951728

Epoch 5: train loss: 0.007181005059816228
valid loss: 0.004911535991040247

Epoch 6: train loss: 0.005116729838390751
valid loss: 0.003934838629424409

Epoch 7: train loss: 0.004172454117668212
valid loss: 0.003158249184005777

Epoch 8: train loss: 0.0033436879387880405
valid loss: 0.0024633239509405497

Epoch 9: train loss: 0.0024183676838536476
valid loss: 0.0017603311830841494

Epoch 10: train loss: 0.001628129033473753
valid loss: 0.001178886987716774

Epoch 11: train loss: 0.001247041985081165
valid loss: 0.0009884663276079166

Epoch 12: train loss: 0.0010405299580971182
valid loss: 0.0008592418369536843

Epoch 13

[I 2025-03-09 17:22:20,494] Trial 0 finished with value: 1.064 and parameters: {'sigma_1': 1.6362639316276848, 'sigma_2': 3.630756580378398, 'sigma_3': 4.264522699896315}. Best is trial 0 with value: 1.064.


0.84
0.224
Updated Best Model!
Confusion Matrix:
[[52540  5591]
 [  114   260]]

Epoch 0: train loss: 0.4316960403813231
valid loss: 0.13099435487242997

Epoch 1: train loss: 0.0939745190686922
valid loss: 0.05401020773528704

Epoch 2: train loss: 0.04124517251610984
valid loss: 0.026669619740317754

Epoch 3: train loss: 0.02205901596175211
valid loss: 0.014210693352942259

Epoch 4: train loss: 0.01027963491513734
valid loss: 0.006026283251834128

Epoch 5: train loss: 0.006019776053975976
valid loss: 0.004480094818367821

Epoch 6: train loss: 0.004808621563308518
valid loss: 0.0036508622995318203

Epoch 7: train loss: 0.003958571678981092
valid loss: 0.0031106067513487217

Epoch 8: train loss: 0.0031043948507938983
valid loss: 0.002174089571506884

Epoch 9: train loss: 0.0020640459801244014
valid loss: 0.0013809389556895513

Epoch 10: train loss: 0.0013407024022747797
valid loss: 0.001007757775448619

Epoch 11: train loss: 0.0010563947601389873
valid loss: 0.0009015469711981306

Epoch 

[I 2025-03-09 17:34:47,749] Trial 1 finished with value: 1.079 and parameters: {'sigma_1': 1.88631940465443, 'sigma_2': 2.5205052948671325, 'sigma_3': 3.982794731726996}. Best is trial 1 with value: 1.079.


...

0.84
0.229

Epoch 0: train loss: 0.46888906262943214
valid loss: 0.15806748883320335

Epoch 1: train loss: 0.09609139601683982
valid loss: 0.04171556153543334

Epoch 2: train loss: 0.029449563689821937
valid loss: 0.01823770576996393

Epoch 3: train loss: 0.013410506090665696
valid loss: 0.006574305000109344

Epoch 4: train loss: 0.005282206290105917
valid loss: 0.0034592973349534505

Epoch 5: train loss: 0.003375678313637854
valid loss: 0.0023174828595273157

Epoch 6: train loss: 0.002050915381452247
valid loss: 0.0013560849559350865

Epoch 7: train loss: 0.0012788230552267725
valid loss: 0.0009309818764609066

Epoch 8: train loss: 0.0009512648192698663
valid loss: 0.0007644816185347735

Epoch 9: train loss: 0.000767925924082406
valid loss: 0.0006254164925382009

Epoch 10: train loss: 0.0006484784593284361
valid loss: 0.0005098258206223856

Epoch 11: train loss: 0.0005549778036472038
valid loss: 0.0004745666980469864

Epoch 12: train loss: 0.0004860812698644061
valid loss: 0.000372863

[I 2025-03-09 19:01:28,985] Trial 8 finished with value: 1.07 and parameters: {'sigma_1': 4.206830428984571, 'sigma_2': 0.7564289647648477, 'sigma_3': 0.5156472819038445}. Best is trial 4 with value: 1.083.


0.844
0.226

Epoch 0: train loss: 0.4310597065643872
valid loss: 0.1223132147733631

Epoch 1: train loss: 0.0843896631553883
valid loss: 0.043086710272998104

Epoch 2: train loss: 0.036961578292127546
valid loss: 0.026046051894177177

Epoch 3: train loss: 0.022904992253158105
valid loss: 0.015430852786072941

Epoch 4: train loss: 0.01232080289383506
valid loss: 0.0071101059498458775

Epoch 5: train loss: 0.006710742967760628
valid loss: 0.00480177144448736

Epoch 6: train loss: 0.005064667870306604
valid loss: 0.003775572551088662

Epoch 7: train loss: 0.003972304535960248
valid loss: 0.002906068165969775

Epoch 8: train loss: 0.002920006858483206
valid loss: 0.002021639502245953

Epoch 9: train loss: 0.0019161530359012447
valid loss: 0.001369488009574818

Epoch 10: train loss: 0.001411834892102991
valid loss: 0.0010985547693222353

Epoch 11: train loss: 0.0011771738423863925
valid loss: 0.0009565102796851469

Epoch 12: train loss: 0.0010200987679502762
valid loss: 0.000838029481593997

[I 2025-03-09 19:12:49,586] Trial 9 finished with value: 1.075 and parameters: {'sigma_1': 2.9416111497586295, 'sigma_2': 4.261203276586734, 'sigma_3': 1.7831452259956362}. Best is trial 4 with value: 1.083.


0.849
0.226
Best trial parameters:
{'sigma_1': 4.0062080556095605, 'sigma_2': 2.852471541404059, 'sigma_3': 1.6364567948104045}


In [15]:
# Gumbel activation class
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.5, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.5, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.5, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-09 19:15:33,672] A new study created in memory with name: no-name-b576266b-7f89-450a-93ea-048209e896ee



Epoch 0: train loss: 0.42928094353462726
valid loss: 0.11875213556032363

Epoch 1: train loss: 0.07404455492525837
valid loss: 0.03730664134758418

Epoch 2: train loss: 0.03274948201204577
valid loss: 0.020704992853430776

Epoch 3: train loss: 0.015911910848524672
valid loss: 0.009321471761728897

Epoch 4: train loss: 0.007609348782662295
valid loss: 0.00525966769649235

Epoch 5: train loss: 0.005362784443912853
valid loss: 0.004259464091270185

Epoch 6: train loss: 0.00425422794108117
valid loss: 0.003250206924231896

Epoch 7: train loss: 0.0031219769723759695
valid loss: 0.0024245494251429545

Epoch 8: train loss: 0.0022748180301806553
valid loss: 0.0019004325835349782

Epoch 9: train loss: 0.0018756824248847882
valid loss: 0.0015990069704104846

Epoch 10: train loss: 0.0016386426855493016
valid loss: 0.0014096954901692523

Epoch 11: train loss: 0.001456519231847069
valid loss: 0.0012552913521350327

Epoch 12: train loss: 0.0013168170087904905
valid loss: 0.0011884063645138767

Epoc

[I 2025-03-09 19:27:05,325] Trial 0 finished with value: 1.078 and parameters: {'sigma_1': 0.5926482554077268, 'sigma_2': 1.2827310239521559, 'sigma_3': 2.709730286624869}. Best is trial 0 with value: 1.078.


0.854
0.224
Updated Best Model!
Confusion Matrix:
[[52543  5588]
 [  111   263]]

Epoch 0: train loss: 0.42346212644678455
valid loss: 0.13237607392782721

Epoch 1: train loss: 0.10229917039051224
valid loss: 0.06357947806191575

Epoch 2: train loss: 0.04904010953705751
valid loss: 0.02958298335717024

Epoch 3: train loss: 0.025962843881104404
valid loss: 0.019634850987849013

Epoch 4: train loss: 0.01727776170038684
valid loss: 0.011190743941834063

Epoch 5: train loss: 0.008123029485508713
valid loss: 0.005055771311337912

Epoch 6: train loss: 0.005264965088567901
valid loss: 0.0039340636893964364

Epoch 7: train loss: 0.0042713908389581095
valid loss: 0.0032478538592463062

Epoch 8: train loss: 0.003473147921184929
valid loss: 0.0026114113695023815

Epoch 9: train loss: 0.002542210824021457
valid loss: 0.0017463149633920177

Epoch 10: train loss: 0.0016286908079630536
valid loss: 0.0011694729030437645

Epoch 11: train loss: 0.0012148344848326355
valid loss: 0.0009700890921203857

Ep

[I 2025-03-09 19:38:50,363] Trial 1 finished with value: 1.074 and parameters: {'sigma_1': 2.0564368178376604, 'sigma_2': 2.7698901725846325, 'sigma_3': 2.3979021180791094}. Best is trial 0 with value: 1.078.


...

0.851
0.234
Updated Best Model!
Confusion Matrix:
[[52549  5582]
 [  105   269]]

Epoch 0: train loss: 0.43264469112595455
valid loss: 0.12573461335539166

Epoch 1: train loss: 0.09121870394461355
valid loss: 0.05617979310982214

Epoch 2: train loss: 0.045255851668321724
valid loss: 0.030581841584233965

Epoch 3: train loss: 0.02732384443927706
valid loss: 0.019146917718640768

Epoch 4: train loss: 0.016398670691836547
valid loss: 0.010546976775745225

Epoch 5: train loss: 0.008758600201790735
valid loss: 0.005362564177491544

Epoch 6: train loss: 0.005425915304334553
valid loss: 0.004020666420749832

Epoch 7: train loss: 0.004390147301732575
valid loss: 0.0033277570373058075

Epoch 8: train loss: 0.003706985626153356
valid loss: 0.002795370887135425

Epoch 9: train loss: 0.002971405136473762
valid loss: 0.0020634352205361906

Epoch 10: train loss: 0.001926407380693772
valid loss: 0.001212869714239429

Epoch 11: train loss: 0.0011116557593586949
valid loss: 0.0008096688366278782

Epoch

[I 2025-03-09 21:03:16,853] Trial 8 finished with value: 1.092 and parameters: {'sigma_1': 4.684630151003316, 'sigma_2': 2.054853992448888, 'sigma_3': 2.927769384042177}. Best is trial 8 with value: 1.092.


0.859
0.233
Updated Best Model!
Confusion Matrix:
[[52559  5572]
 [   95   279]]

Epoch 0: train loss: 0.43911636429153256
valid loss: 0.14844780504052105

Epoch 1: train loss: 0.08816837050540668
valid loss: 0.044573980547447024

Epoch 2: train loss: 0.03420103107709046
valid loss: 0.02221151788418736

Epoch 3: train loss: 0.0186156475975269
valid loss: 0.011904914090358983

Epoch 4: train loss: 0.009186604991555214
valid loss: 0.005745162158349498

Epoch 5: train loss: 0.0056518804280614445
valid loss: 0.004192552479910232

Epoch 6: train loss: 0.004258981130925501
valid loss: 0.003149616377825131

Epoch 7: train loss: 0.0030595068078591394
valid loss: 0.0020970384119577256

Epoch 8: train loss: 0.0019451812602268856
valid loss: 0.0013567965611296356

Epoch 9: train loss: 0.0013879453920292708
valid loss: 0.0010873044602753585

Epoch 10: train loss: 0.001148572037674052
valid loss: 0.0010038968305742175

Epoch 11: train loss: 0.0009747699443869699
valid loss: 0.0008196048030777341

E

[I 2025-03-09 21:15:51,539] Trial 9 finished with value: 1.08 and parameters: {'sigma_1': 1.527931202447736, 'sigma_2': 2.214343320800017, 'sigma_3': 2.58894950631338}. Best is trial 8 with value: 1.092.


0.847
0.233
Best trial parameters:
{'sigma_1': 4.684630151003316, 'sigma_2': 2.054853992448888, 'sigma_3': 2.927769384042177}


In [16]:
# Gumbel activation class
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.5, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.5, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.5, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-09 21:43:20,686] A new study created in memory with name: no-name-dc30f1fc-762f-4b1e-be5a-5f981eb5ed74



Epoch 0: train loss: 0.4317359439458833
valid loss: 0.13925358523599438

Epoch 1: train loss: 0.09154051273035502
valid loss: 0.04743162417590944

Epoch 2: train loss: 0.0335934008231269
valid loss: 0.02101020005638482

Epoch 3: train loss: 0.018008824284713766
valid loss: 0.012688177952268084

Epoch 4: train loss: 0.010740846245911108
valid loss: 0.0069090299852843816

Epoch 5: train loss: 0.006628952299184157
valid loss: 0.004829616963232313

Epoch 6: train loss: 0.005195069047097067
valid loss: 0.003916486677499946

Epoch 7: train loss: 0.0043456169258425615
valid loss: 0.0033325160592743526

Epoch 8: train loss: 0.0036102642701853274
valid loss: 0.002646830861451603

Epoch 9: train loss: 0.00270678177015165
valid loss: 0.0018671036532525319

Epoch 10: train loss: 0.001738556514413543
valid loss: 0.0012592160608301377

Epoch 11: train loss: 0.001269813836574462
valid loss: 0.0010002773103070105

Epoch 12: train loss: 0.001059789966645403
valid loss: 0.0008215790344440363

Epoch 13:

[I 2025-03-09 21:54:43,831] Trial 0 finished with value: 1.077 and parameters: {'sigma_1': 3.546072587742912, 'sigma_2': 4.50525716008578, 'sigma_3': 2.982145437118613}. Best is trial 0 with value: 1.077.


0.851
0.226
Updated Best Model!
Confusion Matrix:
[[52550  5581]
 [  104   270]]

Epoch 0: train loss: 0.436173021836103
valid loss: 0.12880687647259953

Epoch 1: train loss: 0.08451280131202411
valid loss: 0.04135142690782013

Epoch 2: train loss: 0.03860016281486811
valid loss: 0.02793900343828677

Epoch 3: train loss: 0.023466930774540904
valid loss: 0.015106204172464017

Epoch 4: train loss: 0.011416394644137294
valid loss: 0.006280273656091078

Epoch 5: train loss: 0.005781327411229678
valid loss: 0.004141264404373783

Epoch 6: train loss: 0.004207913909952404
valid loss: 0.0029973830423404364

Epoch 7: train loss: 0.0029212641533077773
valid loss: 0.001985106884903921

Epoch 8: train loss: 0.0017986662556656796
valid loss: 0.0013049104475210131

Epoch 9: train loss: 0.0012686621826166558
valid loss: 0.0009932950885566919

Epoch 10: train loss: 0.0010249716629344083
valid loss: 0.0008384531174590455

Epoch 11: train loss: 0.0008635433144160915
valid loss: 0.0006798834056953538

Ep

[I 2025-03-09 22:06:58,154] Trial 1 finished with value: 1.079 and parameters: {'sigma_1': 2.3622972113216787, 'sigma_2': 2.1671385319210295, 'sigma_3': 4.871919538650785}. Best is trial 1 with value: 1.079.


...

0.849
0.231

Epoch 0: train loss: 0.4297922092630576
valid loss: 0.14735899339608155

Epoch 1: train loss: 0.08506092590673418
valid loss: 0.04127657057442626

Epoch 2: train loss: 0.03588257978686326
valid loss: 0.02526401822338176

Epoch 3: train loss: 0.02106159785406283
valid loss: 0.013414250773281022

Epoch 4: train loss: 0.010662049572401988
valid loss: 0.0068007268019203765

Epoch 5: train loss: 0.006884360553502652
valid loss: 0.0051631304532126655

Epoch 6: train loss: 0.00548105857719669
valid loss: 0.004212648305110633

Epoch 7: train loss: 0.004488395987880767
valid loss: 0.003405445108643117

Epoch 8: train loss: 0.003612354848632699
valid loss: 0.0026440603107651646

Epoch 9: train loss: 0.0026061624275426063
valid loss: 0.0017760925737967908

Epoch 10: train loss: 0.0016906404163600276
valid loss: 0.0012327889039669736

Epoch 11: train loss: 0.0012440912308768826
valid loss: 0.0009940195529367978

Epoch 12: train loss: 0.0010232594633868043
valid loss: 0.000804129114871

[I 2025-03-09 23:32:44,315] Trial 8 finished with value: 1.066 and parameters: {'sigma_1': 3.0328507214115445, 'sigma_2': 4.350092037999997, 'sigma_3': 3.003553902132397}. Best is trial 5 with value: 1.088.


0.846
0.22

Epoch 0: train loss: 0.42030491918524865
valid loss: 0.12377371645934594

Epoch 1: train loss: 0.0888737367918398
valid loss: 0.05555944388803563

Epoch 2: train loss: 0.04353229038196529
valid loss: 0.027506342035580854

Epoch 3: train loss: 0.02293903816288362
valid loss: 0.014800075730665134

Epoch 4: train loss: 0.012495046050998295
valid loss: 0.007832357994296024

Epoch 5: train loss: 0.006724565771602479
valid loss: 0.00467966502797001

Epoch 6: train loss: 0.004781841541605736
valid loss: 0.003534445883019171

Epoch 7: train loss: 0.0036749832762180902
valid loss: 0.002670954815697312

Epoch 8: train loss: 0.002544977737703747
valid loss: 0.0017340025841640157

Epoch 9: train loss: 0.0015782703224606954
valid loss: 0.0011195430576946342

Epoch 10: train loss: 0.0011284579028754743
valid loss: 0.0008683830052754165

Epoch 11: train loss: 0.0009153720248427356
valid loss: 0.0007243511139140388

Epoch 12: train loss: 0.0007775211200083785
valid loss: 0.0005998783139734

[I 2025-03-09 23:45:04,584] Trial 9 finished with value: 1.084 and parameters: {'sigma_1': 4.605373512236235, 'sigma_2': 2.595498831640305, 'sigma_3': 3.4570148471575535}. Best is trial 5 with value: 1.088.


0.851
0.233
Best trial parameters:
{'sigma_1': 2.3681725439938175, 'sigma_2': 2.965129182885047, 'sigma_3': 0.7939836079333116}


In [10]:
import optuna
class Gumbel(nn.Module):
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma
    
    def forward(self, input):
        output = input * (1-torch.exp(-torch.exp(input / self.sigma)))
        return output

class SimpleAutoencoder(torch.nn.Module):
    
        def __init__(self, input_size, intermediate_size, code_size, trial):
            super(SimpleAutoencoder, self).__init__()
            # parameters
            self.input_size = input_size
            self.intermediate_size = intermediate_size           
            self.code_size  = code_size
            
            self.Gumbel1 = Gumbel(trial.suggest_float("sigma_1", 0.5, 5.0))
            self.Gumbel2 = Gumbel(trial.suggest_float("sigma_2", 0.5, 5.0))
            self.Gumbel3 = Gumbel(trial.suggest_float("sigma_3", 0.5, 5.0))
            #encoder
            self.fc1 = torch.nn.Linear(self.input_size, self.intermediate_size)
            self.fc2 = torch.nn.Linear(self.intermediate_size, self.code_size)
            
            #decoder 
            self.fc3 = torch.nn.Linear(self.code_size, self.intermediate_size)            
            self.fc4 = torch.nn.Linear(self.intermediate_size, self.input_size)
            
            
        def forward(self, x):
            
            hidden = self.fc1(x)
            hidden = self.Gumbel1(hidden)
            
            code = self.fc2(hidden)
            code = self.Gumbel2(code)
 
            hidden = self.fc3(code)
            hidden = self.Gumbel3(hidden)
            
            output = self.fc4(hidden)
            #linear activation in final layer)            
            
            return output

best_model = None
best_model_state = None
best_target = float('-inf')  # 初始设为一个很大的值

def objective(trial):
    global best_model, best_model_state, best_target
    criterion = torch.nn.MSELoss().to(DEVICE)
    model = SimpleAutoencoder(len(input_features), 100,20,trial).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    model,training_execution_time,train_losses,valid_losses = training_loop(model,
                                                                        training_generator,
                                                                        valid_generator,
                                                                        optimizer,
                                                                        criterion,
                                                                        max_epochs=50,
                                                                        apply_early_stopping=False,
                                                                        verbose=True)
    losses = per_sample_mse(model, valid_generator)
    predictions_df=valid_df
    predictions_df['predictions']=losses
    
    performances = performance_assessment(predictions_df, top_k_list=[100])
    
    auc_roc = performances['AUC ROC'].iloc[0]  # AUC ROC 指标值
    card_precision = performances['Card Precision@100'].iloc[0]  # Card Precision@100 指标值
    
    # 合并目标：加权求和（可以根据需求调整权重）
    target_value = auc_roc + card_precision
    print(auc_roc)
    print(card_precision)

    if target_value > best_target:
        best_target = target_value
        # 保存当前最优模型的权重
        best_model_state = model.state_dict()  # 保存当前模型的状态
        print("Updated Best Model!")
        threshold = np.percentile(losses, 90)  # 90th percentile threshold for anomaly detectio
        y_pred = np.where(losses > threshold, 1, 0)
        cm = confusion_matrix(y_valid, y_pred)
        print(f"Confusion Matrix:\n{cm}")
        
    return target_value
 
study = optuna.create_study(direction="maximize")  # 最大化目标值
study.optimize(objective, n_trials=10)
best_trial = study.best_trial

# 打印最优试验中的参数（包括所有的 sigma 值）
print("Best trial parameters:")
print(best_trial.params)

[I 2025-03-10 08:36:46,366] A new study created in memory with name: no-name-6bb5c4b8-a206-40ed-8201-21b5ac6b4816



Epoch 0: train loss: 0.43543793253928254
valid loss: 0.1336579922444182

Epoch 1: train loss: 0.09085325467049449
valid loss: 0.04226688111781097

Epoch 2: train loss: 0.03195247499460985
valid loss: 0.021472080063095183

Epoch 3: train loss: 0.017933389082921393
valid loss: 0.012159633704669815

Epoch 4: train loss: 0.00920788340544752
valid loss: 0.004671906391820295

Epoch 5: train loss: 0.003971672710276122
valid loss: 0.0027050744800680812

Epoch 6: train loss: 0.002476676299054635
valid loss: 0.0016050556839362316

Epoch 7: train loss: 0.001520851890095305
valid loss: 0.0010966966662790036

Epoch 8: train loss: 0.0011509918168932378
valid loss: 0.0009110931485835361

Epoch 9: train loss: 0.0009508760182494583
valid loss: 0.000769299040090476

Epoch 10: train loss: 0.0008158025706416605
valid loss: 0.0007071105195552904

Epoch 11: train loss: 0.0007120468855740228
valid loss: 0.0005818114635220327

Epoch 12: train loss: 0.0006301582710054858
valid loss: 0.0004834804758671264

Epo

[I 2025-03-10 08:48:30,837] Trial 0 finished with value: 1.076 and parameters: {'sigma_1': 1.9918860023568121, 'sigma_2': 1.1194902122654604, 'sigma_3': 2.753869035429141}. Best is trial 0 with value: 1.076.


0.849
0.227
Updated Best Model!
Confusion Matrix:
[[52541  5590]
 [  113   261]]

Epoch 0: train loss: 0.42406316513968473
valid loss: 0.13292015375661068

Epoch 1: train loss: 0.09154758923268569
valid loss: 0.04934902166733976

Epoch 2: train loss: 0.03813341136017447
valid loss: 0.02514088727187752

Epoch 3: train loss: 0.02230287029854882
valid loss: 0.01626400056452862

Epoch 4: train loss: 0.013293178273493115
valid loss: 0.008171965483107854

Epoch 5: train loss: 0.00685721457211112
valid loss: 0.004637433911286946

Epoch 6: train loss: 0.00489177870202229
valid loss: 0.0036797782902407354

Epoch 7: train loss: 0.0038843745148489624
valid loss: 0.0028595113544088733

Epoch 8: train loss: 0.002785058223097505
valid loss: 0.0018772710984969725

Epoch 9: train loss: 0.0017010765721692915
valid loss: 0.001190949402001427

Epoch 10: train loss: 0.0012157338740500883
valid loss: 0.0009170426548168552

Epoch 11: train loss: 0.0009987285722905849
valid loss: 0.0007666519751436398

Epoch

[I 2025-03-10 09:00:30,725] Trial 1 finished with value: 1.097 and parameters: {'sigma_1': 3.586188297253091, 'sigma_2': 2.6303864362978, 'sigma_3': 2.113898402138476}. Best is trial 1 with value: 1.097.


...

0.839
0.236

Epoch 0: train loss: 0.43502774686476015
valid loss: 0.12631080327301078

Epoch 1: train loss: 0.08571802734274481
valid loss: 0.04336296008095715

Epoch 2: train loss: 0.034177822532776896
valid loss: 0.021262464188853573

Epoch 3: train loss: 0.015917786401936904
valid loss: 0.008266257685023905

Epoch 4: train loss: 0.006508513524270052
valid loss: 0.004346457310402613

Epoch 5: train loss: 0.0045932883652279695
valid loss: 0.0033822360594853596

Epoch 6: train loss: 0.0035855233691317557
valid loss: 0.0026058419237976966

Epoch 7: train loss: 0.0025473669652464706
valid loss: 0.0016930999404806745

Epoch 8: train loss: 0.001576676238110079
valid loss: 0.0010954153738646194

Epoch 9: train loss: 0.001081876188570484
valid loss: 0.0008287846409115187

Epoch 10: train loss: 0.0008605480813786359
valid loss: 0.0006774229623658501

Epoch 11: train loss: 0.000734559352712013
valid loss: 0.0005667629031479725

Epoch 12: train loss: 0.0006310828332416401
valid loss: 0.00057399

[I 2025-03-10 10:23:28,814] Trial 8 finished with value: 1.071 and parameters: {'sigma_1': 2.5975546883808627, 'sigma_2': 1.2358002938593695, 'sigma_3': 2.9657894734929777}. Best is trial 1 with value: 1.097.


0.842
0.229

Epoch 0: train loss: 0.4242882072412717
valid loss: 0.14031722572494726

Epoch 1: train loss: 0.08696900479082395
valid loss: 0.051088212208588266

Epoch 2: train loss: 0.03853192699923542
valid loss: 0.024806660658023397

Epoch 3: train loss: 0.020897989365206336
valid loss: 0.013914355644803556

Epoch 4: train loss: 0.011419655999567638
valid loss: 0.0067856309736971975

Epoch 5: train loss: 0.005702638707700654
valid loss: 0.0037778275635836473

Epoch 6: train loss: 0.003881278722039486
valid loss: 0.0027297163145525044

Epoch 7: train loss: 0.002653349807361732
valid loss: 0.0017089770092817197

Epoch 8: train loss: 0.0016147208491444453
valid loss: 0.0010933998489340187

Epoch 9: train loss: 0.001115952145941974
valid loss: 0.0008477322742612676

Epoch 10: train loss: 0.0008945903085514076
valid loss: 0.0007032878358328023

Epoch 11: train loss: 0.0007582315422744128
valid loss: 0.0005866070465811991

Epoch 12: train loss: 0.000653641299456674
valid loss: 0.0005235647

[I 2025-03-10 10:34:52,353] Trial 9 finished with value: 1.08 and parameters: {'sigma_1': 4.570823179068073, 'sigma_2': 2.055607282496501, 'sigma_3': 4.649056671167777}. Best is trial 1 with value: 1.097.


0.847
0.233
Best trial parameters:
{'sigma_1': 3.586188297253091, 'sigma_2': 2.6303864362978, 'sigma_3': 2.113898402138476}
